# Rocksalt-Zincblende Classification Problem

In [1]:
import math as m
import itertools as it 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import metrics
from sklearn import model_selection
import time
import multiprocessing as mp
import ray
import pickle
from scipy.sparse import csr_matrix

# Data Inicialization and Feature Space Generation

In [2]:
def inicializace_dat():
    Z = np.array([
                    3, 4, 5, 6, 7, 8, 9, 
                    11, 12, 13, 14, 15, 16, 
                    17, 19, 20, 29, 30, 31, 32, 
                    33, 34, 35, 37, 38, 47, 48, 
                    49, 50, 51, 52, 53, 55, 56
    ])
    # atomove cislo prvku A, 34 hodnot
    Prvky = np.array([
                 'Li', 'Be', 'B ', 'C ', 'N ',
                 'O ', 'F ', 'Na', 'Mg', 'Al', 'Si',
                 'P ', 'S ', 'Cl', 'K ', 'Ca', 'Cu',
                 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br',
                 'Rb', 'Sr', 'Ag', 'Cd', 'In', 'Sn',
                 'Sb', 'Te', 'I ', 'Cs', 'Ba'
    ])
    # prvky prislusejici danemu atomovemu cislu, 34 hodnot
    IP = np.array([
                  -5.329, -9.459, -8.190, -10.852, -13.585,
                  -16.433, -19.404, -5.223, -8.037,
                  -5.780, -7.758, -9.751, -11.795,
                  -13.902, -4.433, -6.428, -8.389, -10.136,
                  -5.818, -7.567, -9.262, -10.946, -12.650,
                  -4.289, -6.032, -8.058, -9.581, -5.537,
                  -7.043, -8.468, -9.867, -11.257, -4.006,
                  -5.516
    ])
    # ionizacni potencial (IP)
    EA =  np.array([
        -0.698, 0.631, -0.107, -0.872, -1.867, -3.006, -4.273, -0.716,
        0.693, -0.313, -0.993, -1.920, -2.845, -3.971, -0.621, 0.304,
        -1.638, 1.081, -0.108, -0.949, -1.839, -2.751, -3.739, -0.590,
        0.343, -1.667, 0.839, -0.256, -1.039, -1.847, -2.666, -3.513,
        -0.570, 0.278
    ])
    # elektronova afinita (EA)
    EN = np.array([
        3.014, 4.414, 4.149, 5.862, 7.726, 9.720, 11.839, 2.969, 3.672,
        3.046, 4.375, 5.835, 7.320, 8.936, 2.527, 3.062, 5.014, 4.527,
        2.963, 4.258, 5.551, 6.848, 8.194, 2.440, 2.844, 4.862, 4.371,
        2.897, 4.041, 5.158, 6.266, 7.385, 2.288, 2.619
    ])
    # Elektronegativita dle Mullikenovy definice (EN)
    HOMO = np.array([
        -2.874, -5.600, -3.715, -5.416, -7.239, -9.197, -11.294, -2.819,
        -4.782, -2.784, -4.163, -5.596, -7.106, -8.700, -2.426, -3.864,
        -4.856, -6.217, -2.732, -4.046, -5.341, -6.654, -8.001, -2.360,
        -3.641, -4.710, -5.952, -2.697, -3.866, -4.991, -6.109, -7.236,
        -2.220, -3.346
    ])

    LUMO = np.array([
        -0.978, -2.098, 2.248, 1.992, 3.057, 2.541, 1.251, -0.718, -1.358,
        0.695, 0.440, 0.183, 0.642, 0.574, -0.697, -2.133, -0.641, -1.194,
        0.130, 2.175, 0.064, 1.316, 0.708, -0.705, -1.379, -0.479, -1.309,
        0.368, 0.008, 0.105, 0.099, 0.213, -0.548, -2.129
    ])


    # The radii at which the radial probability density of the valence s, p, 
    # and d orbital are respectively maximal.
    r_s = np.array([
        1.652, 1.078, 0.805, 0.644, 0.539, 0.462, 0.406, 1.715, 1.330, 1.092,
        0.938, 0.826, 0.742, 0.679, 2.128, 1.757, 1.197, 1.099, 0.994, 0.917,
        0.847, 0.798, 0.749, 2.240, 1.911, 1.316, 1.232, 1.134, 1.057, 1.001,
        0.945, 0.896, 2.464, 2.149
    ])

    r_p = np.array([
        1.995, 1.211, 0.826, 0.630, 0.511, 0.427, 0.371, 2.597, 1.897, 1.393,
        1.134, 0.966, 0.847, 0.756, 2.443, 2.324, 1.680, 1.547, 1.330, 1.162,
        1.043, 0.952, 0.882, 3.199, 2.548, 1.883, 1.736, 1.498, 1.344, 1.232,
        1.141, 1.071, 3.164, 2.632
    ])

    r_d = np.array([
        6.930, 2.877, 1.946, 1.631, 1.540, 2.219, 1.428, 6.566, 3.171, 1.939,
        1.890, 1.771, 2.366, 1.666, 1.785, 0.679, 2.576, 2.254, 2.163, 2.373,
        2.023, 2.177, 1.869, 1.960, 1.204, 2.968, 2.604, 3.108, 2.030, 2.065,
        1.827, 1.722, 1.974, 1.351
    ])


    dE = np.array([
        -0.059, -0.038, -0.033, -0.022, 0.430, 0.506, 0.495, 0.466, 1.713,
        1.020, 0.879, 2.638, -0.146, -0.133, -0.127, -0.115, -0.178, -0.087,
        -0.055, -0.005, 0.072, 0.219, 0.212, 0.150, 0.668, 0.275, -0.146,
        -0.165, -0.166, -0.168, -0.266, -0.369, -0.361, -0.350, -0.019,
        0.156, 0.152, 0.203, 0.102, 0.275, 0.259, 0.241, 0.433, 0.341, 0.271,
        0.158, 0.202, -0.136, -0.161, -0.164, -0.169, -0.221, -0.369, -0.375,
        -0.381, -0.156, -0.044, -0.030, 0.037, -0.087, 0.070, 0.083, 0.113,
        0.150, 0.170, 0.122, 0.080, 0.016, 0.581, -0.112, -0.152, -0.158,
        -0.165, -0.095, -0.326, -0.350, -0.381, 0.808, 0.450, 0.264, 0.136,
        0.087
    ])
    # dE = E(RS) - E(ZB) ... 82 hodnot pro binární sloučeniny
    AB = np.array([
        'Li-F ', 'Li-Cl', 'Li-Br', 'Li-I ', 'Be-O ', 'Be-S ', 'Be-Se', 'Be-Te',
        'B -N ', 'B -P ', 'B -As', 'C -C ', 'Na-F ', 'Na-Cl', 'Na-Br', 'Na-I ',
        'Mg-O ', 'Mg-S ', 'Mg-Se', 'Mg-Te', 'Al-N ', 'Al-P ', 'Al-As', 'Al-Sb',
        'Si-C ', 'Si-Si', 'K -F ', 'K -Cl', 'K -Br', 'K -I ', 'Ca-O ', 'Ca-S ',
        'Ca-Se', 'Ca-Te', 'Cu-F ', 'Cu-Cl', 'Cu-Br', 'Cu-I ', 'Zn-O ', 'Zn-S ',
        'Zn-Se', 'Zn-Te', 'Ga-N ', 'Ga-P ', 'Ga-As', 'Ga-Sb', 'Ge-Ge', 'Rb-F ',
        'Rb-Cl', 'Rb-Br', 'Rb-I ', 'Sr-O ', 'Sr-S ', 'Sr-Se', 'Sr-Te', 'Ag-F ',
        'Ag-Cl', 'Ag-Br', 'Ag-I ', 'Cd-O ', 'Cd-S ', 'Cd-Se', 'Cd-Te', 'In-N ',
        'In-P ', 'In-As', 'In-Sb', 'Sn-Sn', 'B -Sb', 'Cs-F ', 'Cs-Cl', 'Cs-Br',
        'Cs-I ', 'Ba-O ', 'Ba-S ', 'Ba-Se', 'Ba-Te', 'Ge-C ', 'Sn-C ', 'Ge-Si',
        'Sn-Si', 'Sn-Ge'
    ])
    # Z vektorů dat vytvoření dictionary obsahující listy, které mají prvky svoje vypočtené hodnoty
    # Kodovani pomoci stringu nazev prvku (dva charaktery dlouhy!!!!)
    oniers = {}
    for i in range(len(Prvky)):
        oniers[Prvky[i]]= [Z[i], IP[i], EA[i], EN[i], HOMO[i], LUMO[i], r_s[i], r_p[i], r_d[i]]

    # Data jednotlivych dimeru ulozenych v dictionary listů, celkem 82 listů
    # Tyto listy jsou vlastně matice o 8 radcich a dvou sloupcich
    dimers = {} # inicializace
    temp = [] #inicializace temporary listu
    for i in AB: # pro kazdy dimer
        for j in range(1,9): # vytvori list listů osmi dvojic hodnot (bez Z[i])
            temp.append( [ oniers[i[:2]][j] , oniers[i[3:]][j] ] )
            # [IP, EA, EN, HOMO, LUMO, r_s, r_p, r_d]
        dimers[i] = temp # kodovani pomoci nazvu dimeru
        temp = [] # clearing pro dalsi iteraci
    dE = dE.reshape(-1,1) # restrukturalizace dat
    return dimers, AB, dE
    
def feature_space_generation(noise, noised_feature, sigma, dimers, AB, tier0, tier1, tier2, tier3, tier4, tier5):    
    # Nyni definujeme a nagenerujeme mnoziny moznych deskriptoru
    # Jednotlive mnoziny jsou listy floatovych hodnot ulozene jako dictionary a klic je  nazev dimeru ve formatu '__-__'
    # Brute force definice zakladnich mnozin deskriptorů pro kazdy dimer:
    
    # tier - which tier of the descriptor to include
    # Vektor popisující tvar deskriptorů:
    A1 = {}
    A2 = {}
    A3 = {}
    for i in AB:
        A1[i] = [ dimers[i][0][0] , dimers[i][1][0] , dimers[i][0][1] , dimers[i][1][1] ] 
        A2[i] = [ dimers[i][3][0] , dimers[i][4][0] , dimers[i][3][1] , dimers[i][4][1] ]
        A3[i] = [ dimers[i][5][0] , dimers[i][6][0] , dimers[i][7][0] , dimers[i][5][1] , dimers[i][6][1] , dimers[i][7][1] ]
        
    if noise==True:
        gauss = np.random.normal(1, sigma, 1)[0]
        
        if noised_feature==1 or noised_feature==True:
            for i in AB:
                A1[i][0] = A1[i][0]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==2 or noised_feature==True:
            for i in AB:
                A1[i][1] = A1[i][1]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==3 or noised_feature==True:
            for i in AB:
                A1[i][2] = A1[i][2]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==4 or noised_feature==True:
            for i in AB:
                A1[i][3] = A1[i][3]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==5 or noised_feature==True:
            for i in AB:
                A2[i][0] = A2[i][0]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==6 or noised_feature==True:
            for i in AB:
                A2[i][1] = A2[i][1]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==7 or noised_feature==True:
            for i in AB:
                A2[i][2] = A2[i][2]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==8 or noised_feature==True:
            for i in AB:
                A2[i][3] = A2[i][3]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==9 or noised_feature==True:
            for i in AB:
                A3[i][0] = A3[i][0]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==10 or noised_feature==True:
            for i in AB:
                A3[i][1] = A3[i][1]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==11 or noised_feature==True:
            for i in AB:
                A3[i][2] = A3[i][2]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==12 or noised_feature==True:
            for i in AB:
                A3[i][3] = A3[i][3]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==13 or noised_feature==True:
            for i in AB:
                A3[i][4] = A3[i][4]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==14 or noised_feature==True:
            for i in AB:
                A3[i][5] = A3[i][5]*np.random.normal(1, sigma, 1)[0]
            
        
    DD = []
    DD.append('IP(A)')
    DD.append('EA(A)')
    DD.append('IP(B)') #2
    DD.append('EA(B)') #3

    DD.append('H(A)')
    DD.append('L(A)')
    DD.append('H(B)')
    DD.append('L(B)')

    DD.append('r_s(A)')
    DD.append('r_p(A)')
    DD.append('r_d(A)')
    DD.append('r_s(B)')
    DD.append('r_p(B)')
    DD.append('r_d(B)')
    
    if tier0==True:####
        DD_A1=DD[:4]
        DD_A2=DD[4:8]
        DD_A3=DD[8:14]

    # Generovani jednoduchych deskriptoru
    DD_B1 = []
    DD_B2 = []
    DD_B3 = []

    DD_C3 = []
    DD_D3 = []
    DD_E3 = []
    
    if tier1==True:####
        DD_dvojice = list( it.combinations( DD_A1 , 2 ) )
        for j in DD_dvojice:
            DD_B1.append('|'+j[0]+'-'+j[1]+'|')
            DD_B1.append('|'+j[0]+'+'+j[1]+'|')

        DD_dvojice = list( it.combinations( DD_A2 , 2 ) )
        for j in DD_dvojice:
            DD_B2.append('|'+j[0]+'-'+j[1]+'|')
            DD_B2.append('|'+j[0]+'+'+j[1]+'|')

        DD_dvojice = list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_B3.append('|'+j[0]+'-'+j[1]+'|')
            DD_B3.append('|'+j[0]+'+'+j[1]+'|')

    DD = DD + DD_B1 + DD_B2 + DD_B3
    
    if tier1==True:####
        for j in DD_A3:
            DD_C3.append(j+'^2')
            
    if tier2==True:####
        DD_dvojice = list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_C3.append('('+j[0]+'+'+j[1]+')^2')
    
    if tier1==True:####
        for j in DD_A3:
            DD_D3.append('exp('+j+')')
            
    if tier2==True:####
        DD_dvojice = list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_D3.append('exp('+j[0]+'+'+j[1]+')')
            
    if tier2==True:####
        for j in DD_A3:
            DD_E3.append('exp('+j+')^2')

    if tier3==True:
        DD_dvojice = list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_E3.append('exp('+j[0]+'+'+j[1]+')^2')

    DD = DD + DD_C3 + DD_D3 + DD_E3


    B1 = {}
    B2 = {}
    B3 = {}
    C3 = {}
    D3 = {}
    E3 = {}
    temp = []

    for i in AB:

        if tier1==True:####
            dvojice = list( it.combinations( A1[i] , 2 ) )
            for j in dvojice:
                temp.append( abs( j[0] - j[1] ) )
                temp.append( abs( j[0] + j[1] ) )

            B1[i] = temp
            temp = []


            dvojice = list( it.combinations( A2[i] , 2 ) )
            for j in dvojice:
                temp.append( abs( j[0] - j[1] ) )
                temp.append( abs( j[0] + j[1] ) )

            B2[i] = temp
            temp = []


            dvojice=list( it.combinations( A3[i] , 2 ) )
            for j in dvojice:
                temp.append( abs( j[0] - j[1] ) )
                temp.append( abs( j[0] + j[1] ) )

            B3[i] = temp
            temp = []
            
        else:
            
            B1[i] = []
            B2[i] = []
            B3[i] = []
            temp = []



        if tier1==True:####
            for j in A3[i]:
                temp.append( (j)**2 )

        if tier2==True:####
            dvojice=list( it.combinations( A3[i] , 2 ) )
            for j in dvojice:
                temp.append( ( j[0] + j[1] )**2 )

        C3[i] = temp
        temp = []

        if tier1==True:####
            for j in A3[i]:
                temp.append( m.exp( j ) )

        if tier2==True:####
            dvojice = list( it.combinations( A3[i] , 2 ) )
            for j in dvojice:
                temp.append( m.exp( j[0] + j[1] ) )

        D3[i] = temp
        temp = []

        if tier2==True:####
            for j in A3[i]:
                temp.append( m.exp( (j)**2 ) )

        if tier3==True:####
            dvojice = list( it.combinations( A3[i] , 2 ) )
            for j in dvojice:
                temp.append( m.exp( ( j[0] + j[1] )**2 ) )

        E3[i] = temp
        temp = []



    G = {}
    temp = []

    DD_G = []


    # A1,B1 ; A2,B2 lomeno A3,C3,D3,E3
    if tier1==True and tier2==False:####                    
        for j in [DD_A1, DD_A2]:
            for l in [DD_A3]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==False:                    
        for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
            for l in [DD_A3]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])
                    
        for j in [DD_A1, DD_A2]:
            for l in [DD_C3[:6], DD_D3[:6]]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==False:            
        for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
            for l in [DD_A3, DD_C3[:6], DD_D3[:6]]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])
                    
        for j in [DD_A1, DD_A2]:
            for l in [DD_C3[6:], DD_D3[6:]]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==False:                    
        for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
            for l in [DD_A3, DD_C3, DD_D3]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

        for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
            for l in [DD_E3[:6]]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])
                    
        for j in [DD_A1, DD_A2]:
            for l in [DD_E3[6:]]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==True:                       
        for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
            for l in [DD_A3, DD_C3, DD_D3, DD_E3]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

# no tiers:
#    for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
#        for l in [DD_C3, DD_D3, DD_E3]:
#            for k in list(it.product(j,l)):
#                DD_G.append(k[0]+'/'+k[1])
                    

    
    
    
    # A3/D3 a A3/E3
    if tier1==True and tier2==True and tier3==False:
        for j in [DD_D3[:6]]:
            for k in list(it.product(DD_A3,j)):
                DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==False:                
        for j in [DD_D3, DD_E3[:6]]:
            for k in list(it.product(DD_A3,j)):
                DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==True:
        for j in [DD_D3, DD_E3]:
            for k in list(it.product(DD_A3,j)):
                DD_G.append(k[0]+'/'+k[1])

    # B3/D3 a B3/E3
    if tier1==True and tier2==True and tier3==True and tier4==False:
        for j in [DD_D3[:6]]:
            for k in list(it.product(DD_B3,j)):
                DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==False:
        for j in [DD_D3, DD_E3[:6]]:
            for k in list(it.product(DD_B3,j)):
                DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==True:  
        for j in [DD_D3, DD_E3]:
            for k in list(it.product(DD_B3,j)):
                DD_G.append(k[0]+'/'+k[1])
    
    
# no tiers:
#    for j in [DD_D3, DD_E3]:
#        for k in list(it.product(DD_A3,j)):
#            DD_G.append(k[0]+'/'+k[1])
#
#    for j in [DD_D3, DD_E3]:
#        for k in list(it.product(DD_B3,j)):
#            DD_G.append(k[0]+'/'+k[1])


    # Problemove:

    # A3/A3
    if tier1==True:####
        DD_dvojice=list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_G.append(j[0]+'/'+j[1])
            DD_G.append(j[1]+'/'+j[0])


    # A3/C3
    if tier2==True:####
        DD_dvojice=list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_G.append(j[0]+'/'+'('+j[1]+')'+'^2')
            DD_G.append(j[1]+'/'+'('+j[0]+')'+'^2')
            #DD_G.append(j[0]+'/'+'('+j[0] +'+'+ j[1]+')^2')
            #DD_G.append(j[1]+'/'+'('+j[0] +'+'+ j[1]+')^2')

    if tier3==True:####
        DD_dvojice=list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            #DD_G.append(j[0]+'/'+'('+j[1]+')'+'^2')
            #DD_G.append(j[1]+'/'+'('+j[0]+')'+'^2')
            DD_G.append(j[0]+'/'+'('+j[0] +'+'+ j[1]+')^2')
            DD_G.append(j[1]+'/'+'('+j[0] +'+'+ j[1]+')^2')


    if tier3==True:####
        DD_trojice=list(it.combinations(DD_A3,3))
        for j in DD_trojice:
            for k in [(0,1,2),(1,0,2),(2,0,1)]:
                DD_G.append(j[k[0]]+'/'+'('+j[k[1]] +'+'+ j[k[2]]+')^2')



    # B3/A3:
    if tier2==True:####
        DD_trojice=list(it.combinations(DD_A3,3))
        for j in DD_trojice:
            for k in [(0,1,2),(2,1,0),(0,2,1)]:
                DD_G.append('|'+j[k[0]] +'-'+ j[k[1]]+'| /'+j[k[2]])
    
    if tier2==True:####
        DD_dvojice=list(it.combinations(DD_A3,2))
        for j in DD_dvojice:
            DD_G.append('|'+j[1]+'-'+j[0]+'| /' + j[1])
            DD_G.append('|'+j[0]+'-'+j[1]+'| /' + j[0])

    # B3/C3
    if tier3==True:####
        DD_trojice=list(it.combinations(DD_A3,3))
        for j in DD_trojice:
            for k in [(0,1,2),(2,1,0),(0,2,1)]:
                DD_G.append('|' + j[k[0]] + '-' +  j[k[1]]+'| /'+j[k[2]]+ '^2')
    
    if tier4==True:####
        DD_dvojice=list(it.combinations(DD_A3,2))
        for j in DD_dvojice:
            DD_G.append('|'+j[0]+'-'+j[1]+'| /'+'('+j[0]+'+'+j[1]+')^2')

    if tier3==True:####
        DD_dvojice=list(it.combinations(DD_A3,2))
        for j in DD_dvojice:
            DD_G.append('|'+j[0]+'-'+j[1]+'| /' + j[0]+'^2')
            DD_G.append('|'+j[1]+'-'+j[0]+'| /'+j[1]+'^2')

    if tier4==True:####
        DD_trojice=list(it.combinations(DD_A3,3))
        for j in DD_trojice:
            DD_G.append('|'+j[0]+'-'+j[1]+'| /'+ '('+j[0]+'+'+j[2]+')^2')
            DD_G.append('|'+j[0]+'-'+j[2]+'| /'+ '('+j[0]+'+'+j[1]+')^2')

    if tier4==True:####
        DD_ctverice = list(it.combinations(DD_A3,4))
        for j in DD_ctverice:
            for k in [(0,1,2,3),(0,2,1,3),(0,3,1,2),(2,1,0,3),(3,1,0,2),(2,3,0,1)]:
                #temp.append(abs(j[k[0]]+j[k[1]])/(j[k[2]]+j[k[3]])**2)
                DD_G.append('|'+j[k[0]]+'-'+j[k[1]]+') /'+'('+j[k[2]]+'+'+j[k[3]]+')^2')


    # Zde je TEST: Ciste podily 1/r navic:
    if tier1==True:####
        for j in DD_A3:
            DD_G.append('1/' + j)

    DD =  DD + DD_G

#####################################################
    temp = []
    for i in AB:
        # A1,B1 ; A2,B2 lomeno A3,C3,D3,E3
        if tier1==True and tier2==False:####
            for j in [A1[i], A2[i]]:
                for l in [A3[i]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
        elif tier1==True and tier2==True and tier3==False:
            for j in [A1[i], A2[i], B1[i], B2[i]]:
                for l in [A3[i]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
            for j in [A1[i], A2[i]]:
                for l in [C3[i][:6], D3[i][:6]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
        elif tier1==True and tier2==True and tier3==True and tier4==False:            
            for j in [A1[i], A2[i], B1[i], B2[i]]:
                for l in [A3[i], C3[i][:6], D3[i][:6]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
            for j in [A1[i], A2[i]]:
                for l in [C3[i][6:], D3[i][6:]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
        elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==False:
            for j in [A1[i], A2[i], B1[i], B2[i]]:
                for l in [A3[i], C3[i], D3[i]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
            for j in [A1[i], A2[i], B1[i], B2[i]]:
                for l in [E3[i][:6]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
            for j in [A1[i], A2[i]]:
                for l in [E3[i][6:]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])

        elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==True:   
            for j in [A1[i], A2[i], B1[i], B2[i]]:
                for l in [A3[i], C3[i], D3[i], E3[i]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
###################################################
                        
                        
###################################################
        # A3/D3 a A3/E3
        if tier1==True and tier2==True and tier3==False:
            for j in [D3[i][:6]]:
                for k in list(it.product(A3[i],j)):
                    temp.append(k[0]/k[1])
                    
        elif tier1==True and tier2==True and tier3==True and tier4==False:
            for j in [D3[i], E3[i][:6]]:
                for k in list(it.product(A3[i],j)):
                    temp.append(k[0]/k[1])
                
        elif tier1==True and tier2==True and tier3==True and tier4==True:
            for j in [D3[i], E3[i]]:
                for k in list(it.product(A3[i],j)):
                    temp.append(k[0]/k[1])
                
        # B3/D3 a B3/E3
        if tier1==True and tier2==True and tier3==True and tier4==False:
            for j in [D3[i][:6]]:
                for k in list(it.product(B3[i],j)):
                    temp.append(k[0]/k[1])
                    
        elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==False:
            for j in [D3[i], E3[i][:6]]:
                for k in list(it.product(B3[i],j)):
                    temp.append(k[0]/k[1])
                    
        elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==True:  
            for j in [D3[i], E3[i]]:
                for k in list(it.product(B3[i],j)):
                    temp.append(k[0]/k[1])

####################################################
                    
        # PROBLEMOVE PODILY:

        # A3/A3 - vsechny podily jenom ne podily X/X = 1:
        if tier1==True:####
            dvojice=list( it.combinations( A3[i] , 2 ) )
            for j in dvojice:
                temp.append(j[0]/j[1])
                temp.append(j[1]/j[0])


        # A3/C3:
        if tier2==True:####
            dvojice=list(it.combinations(A3[i],2))
            for j in dvojice:
                temp.append(j[0]/(j[1])**2)
                temp.append(j[1]/(j[0])**2)
                #temp.append(j[0]/(j[0] + j[1])**2)
                #temp.append(j[1]/(j[0] + j[1])**2)
        
        if tier3==True:####
            dvojice=list(it.combinations(A3[i],2))
            for j in dvojice:
                #temp.append(j[0]/(j[1])**2)
                #temp.append(j[1]/(j[0])**2)
                temp.append(j[0]/(j[0] + j[1])**2)
                temp.append(j[1]/(j[0] + j[1])**2)                
                
                
        if tier3==True:
            trojice=list(it.combinations(A3[i],3))
            for j in trojice:
                for k in [(0,1,2),(1,0,2),(2,0,1)]:
                    temp.append(j[k[0]]/(j[k[1]] + j[k[2]])**2)


        # B3/A3:
        if tier2==True:
            trojice=list(it.combinations(A3[i],3))
            for j in trojice:
                for k in [(0,1,2),(2,1,0),(0,2,1)]:
                    temp.append(abs(j[k[0]] - j[k[1]])/j[k[2]])

        if tier2==True:
            dvojice=list(it.combinations(A3[i],2))
            for j in dvojice:
                temp.append(abs(1-(j[0]/j[1])))
                temp.append(abs(1-(j[1]/j[0])))


        # B3/C3
        if tier3==True:
            trojice=list(it.combinations(A3[i],3))
            for j in trojice:
                for k in [(0,1,2),(2,1,0),(0,2,1)]:
                    temp.append(abs(j[k[0]] - j[k[1]])/j[k[2]]**2)

        if tier4==True:
            dvojice=list(it.combinations(A3[i],2))
            for j in dvojice:
                temp.append(abs(j[0]-j[1])/(j[0]+j[1])**2)

        if tier3==True:
            dvojice=list(it.combinations(A3[i],2))
            for j in dvojice:
                temp.append(abs(j[0]-j[1])/j[0]**2)
                temp.append(abs(j[1]-j[0])/j[1]**2)

        if tier4==True:
            trojice=list(it.combinations(A3[i],3))
            for j in trojice:
                temp.append(abs(j[0]-j[1])/(j[0]+j[2])**2)
                temp.append(abs(j[0]-j[2])/(j[0]+j[1])**2)

        if tier4==True:
            ctverice = list(it.combinations(A3[i],4))
            for j in ctverice:
                for k in [(0,1,2,3),(0,2,1,3),(0,3,1,2),(2,1,0,3),(3,1,0,2),(2,3,0,1)]:
                    #temp.append(abs(j[k[0]]+j[k[1]])/(j[k[2]]+j[k[3]])**2)
                    temp.append(abs(j[k[0]]-j[k[1]])/(j[k[2]]+j[k[3]])**2)






        # Zde je TEST: Ciste podily 1/r navic:
        if tier1==True:####
            for j in A3[i]:
                temp.append(1/j)


        G[i] = temp
        temp = []



    # F1, F2, F3: ... 44 deskriptoru celkem
    if tier3==True:####
        DD_F1 = []
        for j in list(it.combinations(DD_A1[:2],2)):
            for k in list(it.combinations(DD_A1[2:],2)):
                DD_F1.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "+" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F1.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "-" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F1.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "+" + "|" + k[0] + "+" + k[1] + "|" + "|" )
                DD_F1.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "-" + "|" + k[0] + "+" + k[1] + "|" + "|" )

        DD = DD + DD_F1

        DD_F2 = []
        for j in list(it.combinations(DD_A2[:2],2)):
            for k in list(it.combinations(DD_A2[2:],2)):
                DD_F2.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "+" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F2.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "-" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F2.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "+" + "|" + k[0] + "+" + k[1] + "|" + "|" )
                DD_F2.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "-" + "|" + k[0] + "+" + k[1] + "|" + "|" )

        DD = DD + DD_F2

        DD_F3 = []
        for j in list(it.combinations(DD_A3[:3],2)):
            for k in list(it.combinations(DD_A3[3:],2)):
                DD_F3.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "+" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F3.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "-" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F3.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "+" + "|" + k[0] + "+" + k[1] + "|" + "|" )
                DD_F3.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "-" + "|" + k[0] + "+" + k[1] + "|" + "|" )
        
        DD = DD + DD_F3

    # F1, F2, F3: ... 52 deskriptoru celkem

    F1 = {}
    F2 = {}
    F3 = {}

    temp = []
    if tier3==True:####
        for i in AB:
            for j in list(it.combinations(A1[i][:2],2)):
                for k in list(it.combinations(A1[i][2:],2)):
                    temp.append( abs( abs(j[0]-j[1]) + abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]-j[1]) - abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) + abs(k[0]+k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) - abs(k[0]+k[1]) ) )

            F1[i] = temp
            temp = []


        for i in AB:
            for j in list(it.combinations(A2[i][:2],2)):
                for k in list(it.combinations(A2[i][2:],2)):
                    temp.append( abs( abs(j[0]-j[1]) + abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]-j[1]) - abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) + abs(k[0]+k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) - abs(k[0]+k[1]) ) )

            F2[i] = temp
            temp = []


        for i in AB:
            for j in list(it.combinations(A3[i][:3],2)):
                for k in list(it.combinations(A3[i][3:],2)):
                    temp.append( abs( abs(j[0]-j[1]) + abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]-j[1]) - abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) + abs(k[0]+k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) - abs(k[0]+k[1]) ) )

            F3[i] = temp
            temp = []
    else:
        for i in AB:
            F1[i] = []
            F2[i] = []
            F3[i] = []
            temp = []
        
    # 
    Descriptors = len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+ len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]])+len(D3[AB[0]])+len(E3[AB[0]])+len(G[AB[0]])+len(F1[AB[0]])+len(F2[AB[0]])+len(F3[AB[0]])

    # Matice D. Opustíme dictionary a použijeme np.array
    D=np.empty((82,Descriptors),dtype=float)
    for i in range(len(AB)):
        for j in range(len(A1[AB[i]])):
            D[i][j] = A1[AB[i]][j]

        for j in range(len(A2[AB[i]])):
            D[i][j+len(A1[AB[i]])] = A2[AB[i]][j]

        for j in range(len(A3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])] = A3[AB[i]][j]

        for j in range(len(B1[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])] = B1[AB[i]][j]

        for j in range(len(B2[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])] = B2[AB[i]][j]

        for j in range(len(B3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])] = B3[AB[i]][j]

        for j in range(len(C3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])] = C3[AB[i]][j]

        for j in range(len(D3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])] = D3[AB[i]][j]

        for j in range(len(E3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])+len(D3[AB[i]])] = E3[AB[i]][j]

        for j in range(len(G[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])+len(D3[AB[i]])+len(E3[AB[i]])] = G[AB[i]][j]

        for j in range(len(F1[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])+len(D3[AB[i]])+len(E3[AB[i]])+len(G[AB[i]])] = F1[AB[i]][j]

        for j in range(len(F2[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])+len(D3[AB[i]])+len(E3[AB[i]])+len(G[AB[i]])+len(F1[AB[i]])] = F2[AB[i]][j]

        for j in range(len(F3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])+len(D3[AB[i]])+len(E3[AB[i]])+len(G[AB[i]])+len(F1[AB[i]])+len(F2[AB[i]])] = F3[AB[i]][j]


    #print('A1: ', '0 ... ',len(A1[AB[0]])-1)
    #print('A2: ',len(A1[AB[0]]),' ... ',len(A1[AB[0]])+len(A2[AB[0]])-1)
    #print('A3: ',len(A1[AB[0]])+len(A2[AB[0]]), ' ... ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])-1)
    #print('B1: ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]]), ' ... ',  len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[i]])-1)
    #print('B2: ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]]), ' ... ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])-1)
    #print('B3: ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]]), ' ... ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])-1)
    #print('C3: ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]]),' ... ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]])-1)
    #print('D3: ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]]), ' ... ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]])+len(D3[AB[0]])-1)
    #print('E3: ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]])+len(D3[AB[0]]), ' ... ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]])+len(D3[AB[0]])+len(E3[AB[0]])-1)
    
    return D, DD, A1, A2, A3, B1, B2, B3, C3, D3, E3, F1, F2, F3, G

# LASSO+$\ell_{0}$ Minimazition

In [6]:
def LASSO(POCET,LAMBDA_LENGTH,D,dE):
    # LASSO
    # Standardizace dat
    #dE_standard=preprocessing.StandardScaler().fit_transform(dE)
    #D_standard=preprocessing.StandardScaler().fit_transform(D)

    # Normalizace dat
    D_normalized=preprocessing.normalize(D,axis=0)
    #dE_normalized=preprocessing.normalize(dE,axis=0)

    # Hledani nejmensiho lambda, kdy jsou vsechny koeficienty nula. Poté generování 100 hodnot lambda sestupne dle geometricke posloupnosti
    LAMBDA=np.empty(LAMBDA_LENGTH,dtype=float)
    LAMBDA[0]=(1/D.shape[0])*np.max((D_normalized.T).dot(dE))
    print(LAMBDA[0])
    q=m.pow(1000,1/(LAMBDA_LENGTH-1))
    for i in range(1,len(LAMBDA)):
        LAMBDA[i]=LAMBDA[i-1]/q
        #print(LAMBDA[i])
    
    # LASSO fit pro vektor LAMBDA hodnot
    Potencialni=-np.ones((POCET,LAMBDA_LENGTH),dtype=int)
    for j in range(1,LAMBDA_LENGTH): ### BEZIME OD JEDNICKY KVULI ZAOKHROUHLOVANI NA HRANICI NULOVYCH KOEFICIENTU
        lasso = linear_model.Lasso(alpha=LAMBDA[j],fit_intercept=True, normalize=True, max_iter=5e04, warm_start=True, positive=False, copy_X=True, precompute=True, tol=1e-4) # 1e01 1e-1
        lasso.fit(D,dE)

        #print(LAMBDA[j])
        coef = np.absolute(lasso.sparse_coef_.data) # np.absolute? jo nebo ne?
        #print((coef))
        indices = np.array(lasso.sparse_coef_.indices)
        #print(lasso.sparse_coef_)
        
        k=0
        while(k<POCET):
            if len(coef)!=0 and np.max(coef)!=0:
                Potencialni[k,j] = indices[np.argmax(coef)]
                coef[np.argmax(coef)] = 0
                k=k+1
            else:
                break
        
        #k=0
        #while(k<POCET):
        #    if np.max(coef)!=0:
        #        Potencialni[k,j]=lasso.sparse_coef_.indices[np.argmax(coef)]
        #        coef[np.argmax(coef)] = 0
        #        k=k+1
        #    else:
        #        break

    THETA=set() # 31. je #966.... je mozne ze pridanim dalsich sloupcu ktere maji vysokou korelaci nakonec bude zaclenen do danych triceti, ale momentalne neni. Pridanim mnozin F1*, F2*, F3* se dokazalo
    # mezi tricet nejlepsich dat deskriptor cislo 966.
    #for j in range(POCET):
    #    for i in Potencialni[j,:]:
    #        if len(THETA)<POCET and i!=-1:
    #            THETA.add(i)
                
    Potencialni = Potencialni.T
    
    for row in Potencialni:
        #print(row)
        for i in row:
            if len(THETA)<=POCET and i!=-1:
                THETA.add(i)
                
                
    # Ordinary Least Squares
    THETA=list(THETA)
    THETA.sort() # prevedeni setu na list a serazeni vzestupne
    #rutina pro kazdou k-tici. spocitat OLS a porovnat nejlepsi MSE s prave spoctenym.

    # Definujeme nejhorsi mozne MSE, tj. MSE pri coef = 0 a intercept = 0:

    MSE = np.ones((4),dtype = float)*metrics.mean_squared_error(dE,np.zeros(dE.shape,dtype = float)) # ctyr dimenzionalni MSE vektor
    MaxAE = np.zeros((4),dtype = float)
    ND = [ [],[],[],[] ]
    # Specialni rutina pro 1D deskriptor
    for i in THETA:
        OMEGA = D[:,i].reshape(-1, 1)
        ols = linear_model.LinearRegression(fit_intercept = True, normalize = True)
        ols.fit(OMEGA, dE)
        #dE_predicted = np.ones((82,1),dtype = float)*ols.intercept_ + np.dot(OMEGA,ols.coef_.T)
        dE_predicted = ols.predict(OMEGA)
        novy_model_MSE = metrics.mean_squared_error(dE, dE_predicted)

        if novy_model_MSE < MSE[0]:
            MSE[0] = novy_model_MSE
            MaxAE[0] = metrics.max_error(dE,dE_predicted)
            ND[0] = [i, ols.coef_, ols.intercept_, m.sqrt(MSE[0]), MaxAE[0]]

    # Rutina pro 2D, ... ,4D deskriptor
    for j in range(2,5): # 2,3,4
        OMEGA = np.zeros((D.shape[0],j),dtype = float)
        jetice = it.combinations(THETA,j)

        for i in jetice:

            for k in range(j):
                OMEGA[:,k] = D[:,i[k]]

            ols = linear_model.LinearRegression(fit_intercept = True, normalize = True)
            ols.fit(OMEGA,dE)
            #dE_predicted = np.ones((82,1),dtype = float)*ols.intercept_ + np.dot(OMEGA,ols.coef_.T) stejne jako radek pod tim
            dE_predicted = ols.predict(OMEGA)
            novy_model_MSE = metrics.mean_squared_error(dE,dE_predicted)

            if novy_model_MSE < MSE[j-1]:
                MSE[j-1] = novy_model_MSE
                MaxAE[j-1] = metrics.max_error(dE,dE_predicted)
                ND[j-1] = [i, ols.coef_, ols.intercept_, m.sqrt(MSE[j-1]), MaxAE[j-1]]
    return ND

# Training

In [41]:
start_time = time.time()
POCET = 30 # 30,35,40,45
LAMBDA_LENGTH = 100

dimers,AB,dE = inicializace_dat()

(D, DD, A1, A2, A3, B1, B2, B3, C3, D3, E3, F1, F2, F3, G) = feature_space_generation(False, 1, 1, dimers,AB, True, True, True, True, True, True)

ND = LASSO(POCET, LAMBDA_LENGTH, D, dE)


print('| Index deskriptoru ,', 'Vektor koeficientů ,', 'Bias ,', 'RMSE ,', 'MaxAE reziduí| \n')
for i in range(4):
    print(ND[i],'\n')

print('Modely: \n')

print('dE_D1 =','+',round(ND[0][1][0][0],3),'*',DD[ND[0][0]],round(ND[0][2][0],3),'... RMSE =',ND[0][3],'eV', '\n')
print('dE_D2 =','+',round(ND[1][1][0][0],3),'*',DD[ND[1][0][0]],round(ND[1][1][0][1],3),'*',DD[ND[1][0][1]],round(ND[1][2][0],3),'... RMSE =',ND[1][3],'eV', '\n')
print('dE_D3 =','+',round(ND[2][1][0][0],3),'*',DD[ND[2][0][0]],round(ND[2][1][0][1],3),'*',DD[ND[2][0][1]],round(ND[2][1][0][2],3),'*',DD[ND[2][0][2]],round(ND[2][2][0],3),'... RMSE =',ND[2][3],'eV', '\n')
print('dE_D4 =','+',round(ND[3][1][0][0],3),'*',DD[ND[3][0][0]],round(ND[3][1][0][1],3),'*',DD[ND[3][0][1]],'+',round(ND[3][1][0][2],3),'*',DD[ND[3][0][2]],round(ND[3][1][0][3],3),'*',DD[ND[3][0][3]],'+',round(ND[3][2][0],3),'... RMSE =',ND[3][3],'eV', '\n')
elapsed_time = (time.time() - start_time)
print("Elapsed time:", elapsed_time)
#print(DD[ND[0][0]])
#print(DD[ND[1][0][0]],(DD[ND[1][0][1]]))
#print(DD[ND[2][0][0]],(DD[ND[2][0][1]]),(DD[ND[2][0][2]]))
#print(DD[ND[3][0][0]],(DD[ND[3][0][1]]),(DD[ND[3][0][2]]),(DD[ND[3][0][3]]))

# Jak dobre data vzdalenosti dimeru souhlasi s se skutecnymi vzdalenostmi v krystalu? (dimer NaCl vs krystal NaCl)
# Crystalline Atomic Position Estimator (CAPE)

0.0465813419226689
0.043441915379392654
  (0, 819)	0.0038175529431873165
0.040514075677839205
  (0, 819)	0.006830978016593608
  (0, 1105)	0.02124702163334568
0.03778356257303289
  (0, 819)	0.008334334394635204
  (0, 1105)	0.09952177579327454
  (0, 1470)	0.0011425058253567487
0.03523707691278204
  (0, 819)	0.0089280223505542
  (0, 1105)	0.16037210414995304
  (0, 1470)	0.007329704649893038
  (0, 2172)	0.002357490362207977
0.032862215863241964
  (0, 819)	0.009494711536776517
  (0, 1105)	0.21735276534527842
  (0, 1470)	0.0129758155474322
  (0, 2172)	0.004581658790018472
0.030647412500058292
  (0, 819)	0.010023537268558397
  (0, 1105)	0.2704775609397693
  (0, 1470)	0.01823195743808318
  (0, 2172)	0.006670046379160647
0.02858187947086499
  (0, 819)	0.010476750944791242
  (0, 966)	0.00010529105945600439
  (0, 1105)	0.32012474617230846
  (0, 1470)	0.022936070515141523
  (0, 2172)	0.00881692029463018
0.02665555645474148
  (0, 819)	0.010518076297015656
  (0, 966)	0.0012692285609421327
  (0, 1105

In [ ]:
Modely: 

dE_D1 = + 0.055 * |IP(A)+IP(B)|/r_p(A)^2 -0.332 ... RMSE = 0.13779627590238272 eV 

dE_D2 = + 0.113 * |IP(B)-EA(B)|/r_p(A)^2 -1.558 * |r_s(A)-r_p(B)|/exp(r_s(A)) -0.133 ... RMSE = 0.09878859313854685 eV 

dE_D3 = + 1.514 * L(A)/exp(r_d(A)+r_d(B)) 0.104 * |IP(B)-EA(B)|/r_p(A)^2 -1.436 * |r_s(A)-r_p(B)|/exp(r_s(A)) -0.096 ... RMSE = 0.08792895380065499 eV 

dE_D4 = + -0.071 * IP(B)/r_p(A)^2 1.557 * L(A)/exp(r_d(A)+r_d(B)) + 191.474 * r_s(B)/exp(r_s(A)+r_d(B))^2 -1.344 * |r_s(A)-r_p(B)|/exp(r_s(A)) + -0.087 ... RMSE = 0.08266304962176128 eV 

Elapsed time: 35.14220595359802



# Some Poking Around

## A routine for finding an approximation of the lambda value when the number of descriptors changes

In [31]:
i=1
(D, DD, A1, A2, A3, B1, B2, B3, C3, D3, E3, F1, F2, F3, G) = feature_space_generation(dimers,AB)
D_normalized=preprocessing.normalize(D,axis=0)
LAMBDA_LENGTH = 100
LAMBDA=np.empty(LAMBDA_LENGTH,dtype=float)
LAMBDA[0]=(1/D.shape[0])*np.max((D_normalized.T).dot(dE))
q=m.pow(1000,1/(LAMBDA_LENGTH-1))

lasso = linear_model.Lasso(alpha=(LAMBDA[0]-0.0001*0),fit_intercept = True, normalize = True, max_iter=1e05)
lasso.fit(D,dE)
while(len(lasso.sparse_coef_.data)<20):
    lasso = linear_model.Lasso(alpha=(LAMBDA[0]-0.0001*i),fit_intercept = True, normalize = True, max_iter=1e05)
    lasso.fit(D,dE)
    i=i+1
print('Nejvetsi: ', np.max(np.absolute(lasso.sparse_coef_)))
print(np.absolute(lasso.sparse_coef_))
print("Lambda hodnota: ",LAMBDA[0]-0.0001*i, "Pocet iteraci: ",i)

A1:  0 ...  3
A2:  4  ...  7
A3:  8  ...  13
B1:  14  ...  25
B2:  26  ...  37
B3:  38  ...  67
C3:  68  ...  88
D3:  89  ...  109
E3:  110  ...  130
Nejvetsi:  68.45144161234207
  (0, 253)	0.03284668326594066
  (0, 553)	0.3736008229337588
  (0, 600)	0.0289746437931465
  (0, 629)	0.17836123653315425
  (0, 866)	0.011344040469060724
  (0, 1021)	0.35604958671834386
  (0, 2151)	0.026917209678096774
  (0, 2235)	0.009289316452989938
  (0, 2557)	0.04544692441729787
  (0, 2562)	6.033270337070058
  (0, 2717)	0.5231661715188844
  (0, 2764)	0.07088919831887554
  (0, 2769)	1.5846098025152748
  (0, 3022)	1.3054188292933626
  (0, 3097)	0.36794175287696324
  (0, 3110)	3.0067631926163414
  (0, 3399)	68.45144161234207
  (0, 3642)	0.126233021543595
  (0, 3902)	0.03090768629494805
  (0, 4125)	0.008963462506717867
Lambda hodnota:  0.0024813419226688907 Pocet iteraci:  441


## Pearson Correlation Coefficient Calculation of Suspicous Columns

In [28]:
print(DD[3110])
print(DD[3111])
print(DD[3112])
print()
#scipy.stats.pearsonr(D[:,3110],D[:,3111])
for i in it.combinations([D[:,3110], D[:,3111], D[:,3112]],2):
    print(scipy.stats.pearsonr(i[0],i[1]))
print()
for i in it.combinations([D[:,2557], D[:,2558], D[:,2559]],2):
    print(scipy.stats.pearsonr(i[0],i[1]))

|r_s(B)-r_p(B)|/exp(r_d(A)+r_s(B))
|r_s(B)-r_p(B)|/exp(r_d(A)+r_p(B))
|r_s(B)-r_p(B)|/exp(r_d(A)+r_d(B))

(0.9969983364853635, 1.147889135842077e-90)
(0.935572814265117, 6.49166249329293e-38)
(0.9168039305067941, 1.238745872964693e-33)

(0.928003508795371, 4.757784126976563e-36)
(0.8310629510601594, 4.407474079391752e-22)
(0.8872337921934383, 1.3175481682461208e-28)


# Cross Validation

In [109]:
def cross_validation_LASSO(cross_iter, POCET, LAMBDA_LENGTH, D, dE, test_size=7):
    # Vektory co budou drzet hodnoty pro kazdou cross validaci
    RMSE_CV = np.empty((4,cross_iter),dtype = float)
    MaxAE_CV = np.empty((4,cross_iter),dtype = float)
    recovery = np.zeros((4, cross_iter))
    cross_validation_descriptors = np.empty(cross_iter, dtype=object)
    
    for cv in range(cross_iter):
        if test_size==1 and cross_iter==82: # LOOCV
            D_CV = D[[i for i in range(cross_iter) if i!=cv], :]
            X_test = np.array([D[cv, :]])
            dE_CV = dE[[i for i in range(cross_iter) if i!=cv], :]
            y_test = np.array(dE[cv, :])
        else:
            D_CV, X_test, dE_CV, y_test = model_selection.train_test_split(D, dE, test_size=test_size, random_state=cv, shuffle = True)
        
        ND = LASSO(POCET, LAMBDA_LENGTH, D_CV, dE_CV)
        # Pro 1D:
        y_predicted = np.ones((y_test.shape[0],1),dtype = float)*ND[0][2] + np.dot(X_test[:,ND[0][0]].reshape(-1, 1),ND[0][1].T)
        RMSE_CV[0,cv] = np.sqrt(metrics.mean_squared_error(y_test,y_predicted))
        MaxAE_CV[0,cv] = metrics.max_error(y_test,y_predicted)
        
        #Pro 2D, 3D, 4D:
        for j in range(2,5): # 2,3,4
            temporary = np.zeros((y_test.shape[0],j),dtype = float)
            for k in range(j):
                temporary[:,k] = X_test[:,ND[j-1][0][k]]
            y_predicted = np.ones((y_test.shape[0],1),dtype = float)*ND[j-1][2] + np.dot(temporary,ND[j-1][1].T)
            RMSE_CV[j-1,cv] = np.sqrt(metrics.mean_squared_error(y_test, y_predicted))
            MaxAE_CV[j-1,cv] = metrics.max_error(y_test, y_predicted)
        
        # column indices of descriptors
        desc = [819, (966, 2717), (966, 2717, 3097), (2540, 2717, 3022, 3111)]
        
        
        
        if ND[0][0]==desc[0]:
            recovery[0, cv] = 1
        for i in range(1,4):
            if sorted(ND[i][0])==sorted(desc[i]):
                recovery[i, cv] = 1
                
        cross_validation_descriptors[cv] = [(ND[0]), ND[1][0], ND[2][0], ND[3][0]]

    return RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors

## LOOCV

In [9]:
dimers, AB, dE = inicializace_dat()
D, DD, A1, A2, A3, B1, B2, B3, C3, D3, E3, F1, F2, F3, G = feature_space_generation(False, 0, 1, dimers, AB, True, True, True, True, True, True)

D_normalized=preprocessing.normalize(D,axis=0)
LAMBDA_LENGTH = 100
LAMBDA=np.empty(LAMBDA_LENGTH,dtype=float)
LAMBDA[0]=(1/D.shape[0])*np.max((D_normalized.T).dot(dE))
q=m.pow(1000,1/(LAMBDA_LENGTH-1))
for i in range(1,len(LAMBDA)):
    LAMBDA[i]=LAMBDA[i-1]/q

    
lassocv = linear_model.LassoCV(fit_intercept=True, normalize=True, max_iter=1e2, tol=1e-4, copy_X=True)
alphas, coefs, _ = lassocv.path(D, dE, alphas=LAMBDA, max_iter=1e2, tol=1e-4)

/mnt/lustre/helios-home/chmelji6/helios_jupyter_virtual_env/lib64/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:463: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7562598858352706, tolerance: 0.0017140163000000002
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random)
/mnt/lustre/helios-home/chmelji6/helios_jupyter_virtual_env/lib64/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:463: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.637259926825048, tolerance: 0.0017140163000000002
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random)
/mnt/lustre/helios-home/chmelji6/helios_jupyter_virtual_env/lib64/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:463: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.591960363972284,

In [ ]:
print(LAMBDA)
print()
print(alpha)
print()
print(coefs)

In [8]:
start = time.time()
dimers, AB, dE = inicializace_dat()

POCET = 30 # 30,35,40,45
LAMBDA_LENGTH = 100
cross_iter = 82 # 250 500

D, DD, A1, A2, A3, B1, B2, B3, C3, D3, E3, F1, F2, F3, G = feature_space_generation(False, 0, 1, dimers, AB, True, True, True, True, True, True)

RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors = cross_validation_LASSO(cross_iter, POCET, LAMBDA_LENGTH, D, dE, test_size=1)

print("Elapsed: ", (time.time() - start)/60, " min.")
#pickle.dump( sens_data, open( "sens_data_LOOCV.p", "wb" ) )

0.047162548353222
0.0471579663777184
0.04715741711501962
0.04715675907442469
0.04689436681563359
0.04678728638674492
0.04682693709139062
0.046931202706063914
0.04214876311890035
0.04545981282016342
0.0459525798789135
0.03410466180437686
0.04715690122085129
0.0471565519004944
0.047156507443389784
0.04715645477733028
0.0471795906902114
0.04716082874947478
0.047158688297394245
0.04715657746036602
0.047209283601837015
0.04709348340923621
0.04709873843862072
0.047131186079388265
0.04644809976826278
0.04703422563365677
0.047157610730008924
0.047156858390592665
0.04715673254831488
0.04715656485468816
0.04715993705499667
0.047157933376387344
0.04715754560868496
0.04715692485708434
0.047181922240773
0.04713400873414573
0.04713851121676898
0.047141444500085505
0.04714676102331653
0.04709487820745389
0.04710734310344553
0.04712837386347246
0.046867153372698936
0.04701943000117332
0.04706021913023709
0.04712348885621377
0.04709436330429034
0.047156428578380434
0.047156422364006974
0.04715642161226

In [10]:
print("RMSE_CV: ",np.sum(RMSE_CV, axis = 1)/RMSE_CV.shape[1])
print("MaxAE_CV: ",np.sum(MaxAE_CV, axis = 1)/MaxAE_CV.shape[1])

RMSE_CV:  [0.13236105 0.10293266 0.0848045  0.06236542]
MaxAE_CV:  [0.13236105 0.10293266 0.0848045  0.06236542]


In [13]:
RMSE_CV

array([[0.0675161 , 0.03030212, 0.05283023, 0.08344295, 0.20917309,
        0.04708363, 0.068045  , 0.07932441, 0.34576578, 0.09206342,
        0.20164857, 1.00264218, 0.01386802, 0.04470863, 0.06116423,
        0.08491466, 0.22039045, 0.05699752, 0.01149958, 0.05582315,
        0.14320527, 0.11499385, 0.12184567, 0.08171518, 0.21368028,
        0.05313153, 0.03286419, 0.00101199, 0.0096346 , 0.02058792,
        0.16793517, 0.22521928, 0.20841648, 0.18620221, 0.3314157 ,
        0.05707444, 0.07756196, 0.15651701, 0.30583013, 0.10756157,
        0.11096811, 0.11766811, 0.16795722, 0.19441006, 0.13881646,
        0.04923753, 0.47548082, 0.07068044, 0.07523475, 0.07901892,
        0.08153534, 0.07950346, 0.19057118, 0.1894613 , 0.18639569,
        0.25033588, 0.05116802, 0.01744956, 0.07221905, 0.22953866,
        0.0145306 , 0.04328102, 0.09348547, 0.01652034, 0.13129334,
        0.09475242, 0.07180911, 0.07913655, 0.45116053, 0.09384123,
        0.08378827, 0.084665  , 0.08531064, 0.06

# Tier Based Cross Validation 10%,90% <-> 7,75 split

In [32]:
ray.init(num_cpus = 16)

2020-08-12 23:35:05,106	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-08-12 23:35:05,110	INFO resource_spec.py:212 -- Starting Ray with 315.33 GiB memory available for workers and up to 18.63 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-08-12 23:35:05,436	INFO services.py:1078 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '172.20.0.19',
 'redis_address': '172.20.0.19:41983',
 'object_store_address': '/tmp/ray/session_2020-08-12_23-35-05_104750_1623/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-08-12_23-35-05_104750_1623/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-08-12_23-35-05_104750_1623'}

In [73]:
dimers, AB, dE = inicializace_dat()
tier0 = [True, False, False, False, False, False]
tier1 = [True, True, False, False, False, False]
tier2 = [True, True, True, False, False, False]
tier3 = [True, True, True, True, False, False]
tier4 = [True, True, True, True, True, False]
tier5 = [True, True, True, True, True, True]

vysledky_cv = [] # {}

POCET = 50 # 30,35,40,45
LAMBDA_LENGTH = 100
cross_iter = 250 # 250

tiers = [tier0, tier1, tier2, tier3, tier4, tier5]

# Parallel forcycle:
for i in tiers:
    D, DD, A1, A2, A3, B1, B2, B3, C3, D3, E3, F1, F2, F3, G = feature_space_generation(False, 1, 0.1, dimers, AB, *i)
    print("Tier", sum(i)-1)
    #print(D.shape)
    #print(len(DD))
    #vysledky_cv[tiers.index(i)] = [cross_validation_LASSO.remote(cross_iter, POCET, LAMBDA_LENGTH, D, dE)]
    vysledky_cv.append([sum(i)-1, cross_validation_LASSO.remote(cross_iter, POCET, LAMBDA_LENGTH, D, dE)])


Tier 0
Tier 1
Tier 2
Tier 3
Tier 4
Tier 5


In [74]:
from time import time
start = time()
ray.get(vysledky_cv[2][1][1])
print("Elapsed", time()-start)

KeyboardInterrupt: 

In [68]:
# vysledky_cv_real[tiers.index(i)] = [ray.get(vysledky_cv[tiers.index(i)][0][0]), ray.get(vysledky_cv[tiers.index(i)][0][1])]
vysledky_cv_real = {}
for i in tiers:
    vysledky_cv_real[tiers.index(i)] = [ray.get(vysledky_cv[tiers.index(i)][1][0]), ray.get(vysledky_cv[tiers.index(i)][1][1])]

KeyboardInterrupt: 

In [69]:
vysledky_cv_real

{}

In [31]:
ray.shutdown()

In [129]:
tier0 = [True, False, False, False, False, False]
tier1 = [True, True, False, False, False, False]
tier2 = [True, True, True, False, False, False]
tier3 = [True, True, True, True, False, False]
tier4 = [True, True, True, True, True, False]
tier5 = [True, True, True, True, True, True]

tiers = [tier0, tier1, tier2, tier3, tier4, tier5]

#loaded = pickle.load( open( "object_id_clean.p", "rb" ) )
loaded = loaded[0]

for i in tiers:
    print("Tier", sum(i)-1)
    print("RMSE_CV: ", np.round(np.sum(loaded[tiers.index(i)][0][0], axis = 1)/loaded[tiers.index(i)][0][0].shape[1], 2), "eV.")
    print("MaxAE_CV: ", np.round(np.sum(loaded[tiers.index(i)][0][1], axis = 1)/loaded[tiers.index(i)][0][0].shape[1], 2), "eV.")
    print()
    
print("Std of errors:")
for i in tiers:
    print("Tier", sum(i)-1)
    print(np.sqrt(np.sum((loaded[tiers.index(i)][0][0] - np.array([np.sum(loaded[tiers.index(i)][0][0], axis = 1)/loaded[tiers.index(i)][0][0].shape[1]]).T)**2, axis = 1)/loaded[tiers.index(i)][0][0].shape[1]))
    print(np.sqrt(np.sum((loaded[tiers.index(i)][0][1] - np.array([np.sum(loaded[tiers.index(i)][0][1], axis = 1)/loaded[tiers.index(i)][0][1].shape[1]]).T)**2, axis = 1)/loaded[tiers.index(i)][0][1].shape[1]))
    print()

Tier 0


AxisError: axis 1 is out of bounds for array of dimension 0

In [39]:
# LASSO models with tiered feature spaces
dimers, AB, dE = inicializace_dat()
tier0 = [True, False, False, False, False, False]
tier1 = [True, True, False, False, False, False]
tier2 = [True, True, True, False, False, False]
tier3 = [True, True, True, True, False, False]
tier4 = [True, True, True, True, True, False]
tier5 = [True, True, True, True, True, True]

tiers = [tier0, tier1, tier2, tier3, tier4, tier5]

tier_model_output = {}

POCET = 30 # 30,35,40,45
LAMBDA_LENGTH = 100

for i in tiers:
    D, DD, A1, A2, A3, B1, B2, B3, C3, D3, E3, F1, F2, F3, G = feature_space_generation(False, 1, 0.1, dimers, AB, *i)
    print("Tier", sum(i)-1)
    print(D.shape)
    print(len(DD))
    tier_model_output[tiers.index(i)] = LASSO(POCET, LAMBDA_LENGTH, D, dE)
    
pickle.dump( tier_model_output, open( "tier_model.p", "wb" ) )

Tier 0
(82, 14)
14
Tier 1
(82, 164)
164
Tier 2
(82, 596)
596
Tier 3
(82, 1669)
1669
Tier 4
(82, 3566)
3566
Tier 5
(82, 4376)
4376


# Sensitivity Analysis

In [43]:
def LASSO_2D(POCET,LAMBDA_LENGTH,D,dE):
    # LASSO
    # Standardizace dat
    #dE_standard=preprocessing.StandardScaler().fit_transform(dE)
    #D_standard=preprocessing.StandardScaler().fit_transform(D)

    # Normalizace dat
    D_normalized=preprocessing.normalize(D,axis=0)
    #dE_normalized=preprocessing.normalize(dE,axis=0)

    # Hledani nejmensiho lambda, kdy jsou vsechny koeficienty nula. Poté generování 100 hodnot lambda sestupne dle geometricke posloupnosti
    LAMBDA=np.empty(LAMBDA_LENGTH,dtype=float)
    LAMBDA[0]=(1/D.shape[0])*np.max((D_normalized.T).dot(dE))
    q=m.pow(1000,1/(LAMBDA_LENGTH-1))
    for i in range(1,len(LAMBDA)):
        LAMBDA[i]=LAMBDA[i-1]/q
    
    # LASSO fit pro vektor LAMBDA hodnot
    Potencialni = -np.ones((POCET,LAMBDA_LENGTH),dtype=int) #np.zeros((POCET,LAMBDA_LENGTH),dtype=int)-np.ones((POCET,LAMBDA_LENGTH),dtype=int)
    for j in range(1,LAMBDA_LENGTH): ### BEZIME OD JEDNICKY KVULI ZAOKHROUHLOVANI NA HRANICI NULOVYCH KOEFICIENTU
        lasso = linear_model.Lasso(alpha=LAMBDA[j],fit_intercept=True, normalize=True, max_iter=5e03, warm_start=True, positive=False, copy_X=True, precompute=True, tol=1e-3)
        lasso.fit(D,dE)

        coef = np.absolute(lasso.sparse_coef_.data) # np.absolute? jo nebo ne?

        k=0
        while(k<POCET):
            if np.max(coef)!=0:
                Potencialni[k,j]=lasso.sparse_coef_.indices[np.argmax(coef)]
                coef[np.argmax(coef)] = 0
                k=k+1
            else:
                break

    THETA=set() # 31. je #966.... je mozne ze pridanim dalsich sloupcu ktere maji vysokou korelaci nakonec bude zaclenen do danych triceti, ale momentalne neni. Pridanim mnozin F1*, F2*, F3* se dokazalo
    # mezi tricet nejlepsich dat deskriptor cislo 966.
    for j in range(POCET):
        for i in Potencialni[j,:]:
            if len(THETA)<POCET and i!=-1:
                THETA.add(i)
    # Ordinary Least Squares
    THETA=list(THETA)
    THETA.sort() # prevedeni setu na list a serazeni vzestupne
    #rutina pro kazdou k-tici. spocitat OLS a porovnat nejlepsi MSE s prave spoctenym.

    # Definujeme nejhorsi mozne MSE, tj. MSE pri coef = 0 a intercept = 0:

    MSE = metrics.mean_squared_error(dE,np.zeros(dE.shape,dtype = float)) # ctyr dimenzionalni MSE vektor
    MaxAE = 0
    ND = 0
    # Rutina pro 2D deskriptor
    #OMEGA = np.zeros((D.shape[0], 2),dtype = float)
    jetice = it.combinations(THETA, 2)

    for i in jetice:
        #for k in range(2):
        #    OMEGA[:,k] = D[:,i[k]]
        OMEGA = np.array([D[:,i[0]], D[:,i[1]]]).T

        ols = linear_model.LinearRegression(fit_intercept = True, normalize = True)
        ols.fit(OMEGA,dE)
        dE_predicted = ols.predict(OMEGA)
        novy_model_MSE = metrics.mean_squared_error(dE, dE_predicted)

        if novy_model_MSE < MSE:
            MSE = novy_model_MSE
            MaxAE = metrics.max_error(dE,dE_predicted)
            ND = [i, ols.coef_, ols.intercept_, m.sqrt(MSE), MaxAE]
    return ND

In [44]:
def cross_validation_LASSO_2D(cross_iter, POCET, LAMBDA_LENGTH, D, dE, test_size=7):
    print("Starting...")
    # Vektory co budou drzet hodnoty pro kazdou cross validaci
    RMSE_CV = []#np.empty((cross_iter),dtype = float)
    MaxAE_CV = []#np.empty((cross_iter),dtype = float)

    for cv in range(cross_iter):
        if test_size==1 and cross_iter==82: # LOOCV
            D_CV = D[[i for i in range(cross_iter) if i!=cv], :]
            X_test = np.array([D[cv, :]])
            dE_CV = dE[[i for i in range(cross_iter) if i!=cv], :]
            y_test = np.array(dE[cv, :])
        else:
            D_CV, X_test, dE_CV, y_test = model_selection.train_test_split(D, dE, test_size=test_size, random_state=cv, shuffle = True)
        
        ND = LASSO_2D(POCET, LAMBDA_LENGTH, D_CV, dE_CV)
        
        #Pro 2D:
        temporary = np.zeros((y_test.shape[0], 2),dtype = float)
        #for k in range(2):
        #    temporary[:,k] = X_test[:,ND[0][k]]
        #temporary[:,0] = X_test[:,ND[0][0]]
        #temporary[:,1] = X_test[:,ND[0][1]]
        y_predicted = ND[2] + np.dot(np.array([X_test[:,ND[0][0]], X_test[:,ND[0][1]]]).T ,ND[1].T)
        RMSE_CV.append(np.sqrt(metrics.mean_squared_error(y_test, y_predicted)))
        MaxAE_CV.append(metrics.max_error(y_test, y_predicted))
        
        # column indices of descriptors
        desc = [819, (966, 2717), (966, 2717, 3097), (2540, 2717, 3022, 3111)]
        
        recovery = np.zeros(cross_iter)
        
        if sorted(ND[0])==sorted(desc[1]):
            recovery[cv] = 1
                
        cross_validation_descriptors = [ND[0]]
    print("Done.")
    return RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors

### Noise applied to primary features

In [ ]:
sigma = [0.001, 0.01, 0.03, 0.05, 0.1, 0.13, 0.3] # noise
#gauss = np.random.normal(1,sigma,14)
delta = [0, 0.01, 0.03, 0.10, 0.20]


In [46]:
sigma = [0.001]
# LOOCV sensitivity analysis
start = time.time()
dimers, AB, dE = inicializace_dat()

POCET = 30 # 30,35,40,45
LAMBDA_LENGTH = 100
cross_iter = 82 # 250 500
iterace = 1

sens_data = np.empty((len(sigma), 14, iterace), dtype=object) # 14

sens_data = np.empty((len(sigma), 14, iterace), dtype=object) # 14

for sig in sigma: # for noise level
    for feat in range(1): # for every feature , 14
        for i in range(iterace): # do fifty drafts
            D_noised, DD, A1, A2, A3, B1, B2, B3, C3, D3, E3, F1, F2, F3, G = feature_space_generation(False, feat, sig, dimers, AB, True, True, True, True, True, True)
            
            RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors = cross_validation_LASSO_2D(cross_iter, POCET, LAMBDA_LENGTH, D_noised, dE, test_size=1)
            
            sens_data[sigma.index(sig), feat, i] = [RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors]
            
print("Elapsed: ", (time.time() - start)/60, " min.")
#pickle.dump( sens_data, open( "sens_data_LOOCV.p", "wb" ) )

Starting...
Done.
Elapsed:  45.69391427437464  min.


In [128]:
(21.12/60)*7*50/14

8.8

In [ ]:
# L10%OCV
dimers, AB, dE = inicializace_dat()

POCET = 50 # 30,35,40,45
LAMBDA_LENGTH = 100
cross_iter = 50 # 250 500
iterace = 50

sens_data = np.empty((len(sigma), 14, iterace))

for sig in sigma: # for noise level
    for feat in range(1, 14): # for every feature
        for i in range(iterace): # do fifty drafts
            D_noised, DD, A1, A2, A3, B1, B2, B3, C3, D3, E3, F1, F2, F3, G = feature_space_generation(True, feat, sig, dimers, AB, True, True, True, True, True, True)
            
            RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors = cross_validation_LASSO(cross_iter, POCET, LAMBDA_LENGTH, D_noised, dE, test_size=7)
            
            sens_data[sigma[sigma.index(sig)], feat, i] = [RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors]

pickle.dump( sens_data, open( "sens_data_L10OCV.p", "wb" ) )

In [140]:
loaded = pickle.load( open( "vysledky_cv_tier_0.p", "rb" ) )

print("RMSE_CV: ", np.round(np.sum(loaded[0][0], axis = 1)/loaded[0][0].shape[1], 2), "eV.")
print("MaxAE_CV: ", np.round(np.sum(loaded[0][1], axis = 1)/loaded[0][0].shape[1], 2), "eV.")
print()
    
print("Std of errors:")
print(np.sqrt(np.sum((loaded[0][0] - np.array([np.sum(loaded[0][0], axis = 1)/loaded[0][0].shape[1]]).T)**2, axis = 1)/loaded[0][0].shape[1]))
print(np.sqrt(np.sum((loaded[0][1] - np.array([np.sum(loaded[0][1], axis = 1)/loaded[0][1].shape[1]]).T)**2, axis = 1)/loaded[0][1].shape[1]))
print()

RMSE_CV:  [0.27 0.2  0.19 0.2 ] eV.
MaxAE_CV:  [0.54 0.41 0.42 0.41] eV.

Std of errors:
[0.17754724 0.11562685 0.11952612 0.10892632]
[0.49322805 0.32393195 0.31524179 0.28985042]



In [106]:
# RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors
loaded = pickle.load( open( "noise/f_1/sens_data_LOOCV_prim_f_1_n_1.p.1", "rb" ) )

In [2]:
loaded = pickle.load( open( "lasso_tiers/vysledky_cv_tier_"+ str(0) +".p", "rb" ) )
loaded = loaded[0]

In [34]:
for i in range(6):
    loaded = pickle.load( open( "lasso_tiers/vysledky_cv_tier_"+ str(i) +".p", "rb" ) )

    print("RMSE_CV: ", np.round(np.sum(loaded[0][0], axis = 1)/loaded[0][0].shape[1], 3), "eV.")
    print("MaxAE_CV: ", np.round(np.sum(loaded[0][1], axis = 1)/loaded[0][1].shape[1], 3), "eV.")
    #print("Recovery %: ", np.round(np.sum(loaded[0][2], axis = 1)/loaded[0][2].shape[1], 3), "%.") # RECOVERY WRONG BECAUSE DIFFERENT COLUMN INDICES
    print()

    #print("Std of errors:")
    #print(np.sqrt(np.sum((loaded[0][0] - np.array([np.sum(loaded[0][0], axis = 1)/loaded[0][0].shape[1]]).T)**2, axis = 1)/loaded[0][0].shape[1]))
    #print(np.sqrt(np.sum((loaded[0][1] - np.array([np.sum(loaded[0][1], axis = 1)/loaded[0][1].shape[1]]).T)**2, axis = 1)/loaded[0][1].shape[1]))
    print("---------------------------------------------------------------------")

RMSE_CV:  [0.262 0.202 0.19  0.194] eV.
MaxAE_CV:  [0.529 0.404 0.418 0.415] eV.
Recovery %:  [0. 0. 0. 0.] %.

---------------------------------------------------------------------
RMSE_CV:  [0.187 0.192 0.145 0.136] eV.
MaxAE_CV:  [0.38  0.394 0.294 0.266] eV.
Recovery %:  [0. 0. 0. 0.] %.

---------------------------------------------------------------------
RMSE_CV:  [0.162 0.138 0.133 0.102] eV.
MaxAE_CV:  [0.304 0.257 0.267 0.194] eV.
Recovery %:  [0. 0. 0. 0.] %.

---------------------------------------------------------------------
RMSE_CV:  [0.159 0.103 0.079 0.086] eV.
MaxAE_CV:  [0.305 0.18  0.144 0.167] eV.
Recovery %:  [0. 0. 0. 0.] %.

---------------------------------------------------------------------
RMSE_CV:  [0.172 0.129 0.108 0.112] eV.
MaxAE_CV:  [0.34  0.246 0.217 0.233] eV.
Recovery %:  [0. 0. 0. 0.] %.

---------------------------------------------------------------------
RMSE_CV:  [0.172 0.129 0.11  0.093] eV.
MaxAE_CV:  [0.34  0.247 0.22  0.187] eV.
Recovery 

In [8]:
tier=3
delta=0
loaded = pickle.load( open( "lasso_noise_E/sens_data_E_L10OCV_Tier_" + str(tier) + "_delta_"+ str(delta) +".p", "rb" ) )

In [19]:
print(len(loaded))
print(len(loaded[0]))
print(len(loaded[0][0]))
print(loaded[0][0])
#RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors

10
1
4
[array([0.08380071, 0.11366894, 0.1032401 , 0.10864964, 0.11110535,
       0.09084777, 0.1068579 , 0.09094482, 0.09025671, 0.08136976,
       0.09838527, 0.10733885, 0.09912521, 0.08287971, 0.07275471,
       0.10813771, 0.05965471, 0.10819704, 0.15629731, 0.08494381,
       0.0956321 , 0.05643534, 0.12821232, 0.04768037, 0.09977361,
       0.09129503, 0.12414086, 0.11011405, 0.0852749 , 0.07919874,
       0.13068176, 0.07747703, 0.13313453, 0.07814249, 0.1053854 ,
       0.07396585, 0.07162692, 0.11692204, 0.0932008 , 0.11807245,
       0.18891809, 0.13358836, 0.08073796, 0.10431767, 0.13073039,
       0.14219839, 0.12296699, 0.13915998, 0.09413753, 0.08849188]), array([0.14584679, 0.1954411 , 0.18313594, 0.18542067, 0.18917454,
       0.20488755, 0.19653142, 0.1814359 , 0.19574902, 0.12135086,
       0.16185539, 0.19997315, 0.16918556, 0.18312748, 0.11088091,
       0.19311542, 0.10014487, 0.17082144, 0.289839  , 0.14002525,
       0.15186455, 0.09325252, 0.21241032, 0.1043768

In [100]:
# get the tier 3 statistics without noise and use them as comparison with others
dimers, AB, dE = inicializace_dat()

D_tier3, DD_tier3, *_ = feature_space_generation(False, 1, 1, dimers,AB, True, True, True, True, False, False)

D, DD, *_= feature_space_generation(False, 1, 1, dimers,AB, True, True, True, True, True, True)

print(DD_tier3[408], DD_tier3[1145])
print()
print(DD[966], DD[2717])

for tier in [3]:
    loaded = pickle.load( open( "lasso_noise_E/sens_data_E_L10OCV_Tier_" + str(tier) + "_delta_"+ str(0) +".p", "rb" ) )
    nonperturbed = [j for i in [list(loaded[j][0][3]) for j in range(len(loaded))] for j in i]
    for delta in [0,0.01,0.03,0.1,0.2]:
        loaded = pickle.load( open( "lasso_noise_E/sens_data_E_L10OCV_Tier_" + str(tier) + "_delta_"+ str(delta) +".p", "rb" ) )
        print("RMSE_CV " + str(delta) + ": ", np.array([j for i in [list(loaded[j][0][0]) for j in range(len(loaded))] for j in i]).mean() )
        print("AveMaxAE_CV " + str(delta) + ": ", np.array([j for i in [list(loaded[j][0][1]) for j in range(len(loaded))] for j in i]).mean() )
        truest = []
        for descs in [j for i in [list(loaded[j][0][3]) for j in range(len(loaded))] for j in i]:
            truest.append(descs == (408, 1145)) # real 2D with all data
        print("% best all data descriptor: ", sum(truest)/len(truest)*100)
        
        truest = []
        ita = iter(nonperturbed)
        for descs in [j for i in [list(loaded[j][0][3]) for j in range(len(loaded))] for j in i]:
            truest.append(descs == next(ita)) # non perturbed L10OCV 2D
        print("% best 2D L10%OCV descriptor: ", sum(truest)/len(truest)*100)
        
        print()

|IP(B)-EA(B)|/r_p(A)^2 |r_s(A)-r_p(B)|/exp(r_s(A))

|IP(B)-EA(B)|/r_p(A)^2 |r_s(A)-r_p(B)|/exp(r_s(A))
RMSE_CV 0:  0.10200139733256504
AveMaxAE_CV 0:  0.17975852581420876
% best all data descriptor:  94.0
% best 2D L10%OCV descriptor:  100.0

RMSE_CV 0.01:  0.1025875013094045
AveMaxAE_CV 0.01:  0.1802948995258653
% best all data descriptor:  92.0
% best 2D L10%OCV descriptor:  96.0

RMSE_CV 0.03:  0.10598259291609591
AveMaxAE_CV 0.03:  0.18695833323706834
% best all data descriptor:  89.2
% best 2D L10%OCV descriptor:  90.4

RMSE_CV 0.1:  0.1280874225266806
AveMaxAE_CV 0.1:  0.22849688097560783
% best all data descriptor:  64.0
% best 2D L10%OCV descriptor:  62.8

RMSE_CV 0.2:  0.1776109799685648
AveMaxAE_CV 0.2:  0.3185644585350781
% best all data descriptor:  15.0
% best 2D L10%OCV descriptor:  15.0



In [108]:
# get the tier 4 statistics without noise and use them as comparison with others
dimers, AB, dE = inicializace_dat()

D_tier4, DD_tier4, *_ = feature_space_generation(False, 1, 1, dimers,AB, True, True, True, True, True, False)

D, DD, *_= feature_space_generation(False, 1, 1, dimers,AB, True, True, True, True, True, True)

print(DD_tier4[798], DD_tier4[2357])
print()
print(DD[966], DD[2717])

for tier in [4]:
    loaded = pickle.load( open( "lasso_noise_E/sens_data_E_L10OCV_Tier_" + str(tier) + "_delta_"+ str(0) +".p", "rb" ) )
    nonperturbed = [j for i in [list(loaded[j][0][3]) for j in range(len(loaded))] for j in i]
    for delta in [0,0.01,0.03,0.1,0.2]:
        loaded = pickle.load( open( "lasso_noise_E/sens_data_E_L10OCV_Tier_" + str(tier) + "_delta_"+ str(delta) +".p", "rb" ) )
        print("RMSE_CV " + str(delta) + ": ", np.array([j for i in [list(loaded[j][0][0]) for j in range(len(loaded))] for j in i]).mean() )
        print("AveMaxAE_CV " + str(delta) + ": ", np.array([j for i in [list(loaded[j][0][1]) for j in range(len(loaded))] for j in i]).mean() )
        truest = []
        for descs in [j for i in [list(loaded[j][0][3]) for j in range(len(loaded))] for j in i]:
            truest.append(descs == (798, 2357)) # real 2D with all data
        print("% best all data descriptor: ", sum(truest)/len(truest)*100)
        
        truest = []
        ita = iter(nonperturbed)
        for descs in [j for i in [list(loaded[j][0][3]) for j in range(len(loaded))] for j in i]:
            truest.append(descs == next(ita)) # non perturbed L10OCV 2D
        print("% best 2D L10%OCV descriptor: ", sum(truest)/len(truest)*100)
        
        print()

|IP(B)-EA(B)|/r_p(A)^2 |r_s(A)-r_p(B)|/exp(r_s(A))

|IP(B)-EA(B)|/r_p(A)^2 |r_s(A)-r_p(B)|/exp(r_s(A))
RMSE_CV 0:  0.13042500684925187
AveMaxAE_CV 0:  0.2533003468921526
% best all data descriptor:  42.0
% best 2D L10%OCV descriptor:  100.0

RMSE_CV 0.01:  0.12885042961918428
AveMaxAE_CV 0.01:  0.24753623050473914
% best all data descriptor:  38.800000000000004
% best 2D L10%OCV descriptor:  88.0

RMSE_CV 0.03:  0.13069918874943315
AveMaxAE_CV 0.03:  0.2474411007023387
% best all data descriptor:  34.0
% best 2D L10%OCV descriptor:  68.8

RMSE_CV 0.1:  0.14779252961682227
AveMaxAE_CV 0.1:  0.27660651647598294
% best all data descriptor:  11.200000000000001
% best 2D L10%OCV descriptor:  30.2

RMSE_CV 0.2:  0.1845558123809595
AveMaxAE_CV 0.2:  0.3363978441437856
% best all data descriptor:  1.0
% best 2D L10%OCV descriptor:  7.3999999999999995



In [21]:
l10ocv_2D_rec[0]

[[0.3174194939288607,
  0.11366893565665531,
  0.3615956692393133,
  0.14389345649062565,
  0.166619668741127,
  0.09084776719045064,
  0.10685789813011776,
  0.0909448157904827,
  0.09025671260511497,
  0.11204148612330242,
  0.0983852708189361,
  0.10733885325386364,
  0.12506103285514508,
  0.08287971134522835,
  0.08619712856429557,
  0.1081377102906846,
  0.0596547097194745,
  0.1081970356303062,
  0.1562973086835009,
  0.09303498383722932,
  0.10598728876385331,
  0.06576318113662784,
  0.12821232175706987,
  0.05354547515432398,
  0.15236064201616048,
  0.09129502882899501,
  0.1330537302793408,
  0.1750168401622811,
  0.08527490041181791,
  0.07919873820807088,
  0.12078517406888338,
  0.13423393351988044,
  0.13385435142048907,
  0.0781424873918677,
  0.10538540175629704,
  0.08089844762337885,
  0.07162691510913252,
  0.1447135932375611,
  0.1306013639607643,
  0.2590866635723108,
  0.45169030978305,
  0.1537404281333367,
  0.08073796451250094,
  0.10152969109342373,
  0.1307

In [150]:
1 unstable, 2 stable 0.1, 3 stable 0.11, 4 stable 0.1, 5 stable 0.1, 6 stable 0.1, 7 stable 0.1, 8 stable 0.1, 9 stable 0.1, 10 stable 0.11, 11 stable 0.1, 12 stable 0.1, 13 stable  0.1-0.13,  

In [207]:
loaded

[[[0.3164322900665277,
   0.1138361427653082,
   0.361937303960228,
   0.14427577171605518,
   0.1679924974447166,
   0.09093269907684288,
   0.10651811520961967,
   0.09060878523037606,
   0.09007798547799997,
   0.11084506121287308,
   0.09810506776375821,
   0.12371125809229212,
   0.12261618515069594,
   0.08223958850987388,
   0.08491809697406892,
   0.10782339148799276,
   0.05961949113203508,
   0.10765575543653196,
   0.1559020992432558,
   0.09235296317462342,
   0.10583594143222402,
   0.06586008346206829,
   0.12850803780411,
   0.05344368043662166,
   0.1515745540454359,
   0.09129833042292515,
   0.13203109630684937,
   0.17478130173015277,
   0.08507776093319568,
   0.07852460586517881,
   0.09531098038948435,
   0.1342100084773363,
   0.13407764167754058,
   0.07895470826837737,
   0.10533171601454665,
   0.07983862628654724,
   0.07085089518009564,
   0.1442541151649073,
   0.1282178335307774,
   0.25120887049290935,
   0.4581513385393924,
   0.1327812121587469,
   0.08

In [206]:
for noise in range(1,8):
    loaded = pickle.load( open( "lasso_noise_prim_l10ocv/sens_data_L10OCV_prim_f_" + str(1) + "_n_" + str(1) + ".p", "rb" ) )
    li = []
    for i in loaded:
        li.append(i[0])
    vsechno = np.concatenate(li)
    print(vsechno.mean())

0.12968114987976473
0.12968114987976473
0.12968114987976473
0.12968114987976473
0.12968114987976473
0.12968114987976473
0.12968114987976473


In [199]:
import pickle
import numpy as np
primary = ['IP(A)', 'EA(A)', 'IP(B)', 'EA(B)', 'H(A)', 'L(A)', 'H(B)', 'L(B)', 'r_s(A)', 'r_p(A)', 'r_d(A)', 'r_s(B)', 'r_p(B)', 'r_d(B)', "dummy"]
dve_D_all = [primary[0], "Recovery of all data D2 [%]"]
dve_D_loocv = ["", "Recovery of LOOCV D2 [%]"]
dve_D_l10ocv = ["", "Recovery of L10%OCV D2 [%]"]
D_all = []
D_loocv = []
D_l10ocv = []
for feat in range(1,15):
    for num in range(1,8):
        loaded = pickle.load( open( "lasso_noise_prim_loocv/sens_data_LOOCV_prim_f_" + str(feat) + "_n_" + str(num) + ".p", "rb" ) )
        loaded1 = pickle.load( open( "lasso_noise_prim_l10ocv/sens_data_L10OCV_prim_f_" + str(feat) + "_n_" + str(num) + ".p", "rb" ) ) 
        loocv_2D_rec = pickle.load( open( "lasso_noise_prim_loocv/LOOCV_descriptors.p", "rb" ) )[1]
        l10ocv_2D_rec = pickle.load( open( "lasso_noise_prim_l10ocv/L10OCV.p", "rb" ) )[0]
        li = []
        for i in loaded:
            li.append(i[2])
        vsechno = np.concatenate(li)
        #print(vsechno.shape)
        # porovnani s 2D desc vsech dat
        dve_D_all.append(np.round(np.sum(vsechno)/vsechno.shape[0], 2))

        listik = []
        for i in range(82):
            listik.append(loaded[0][3][i] == loocv_2D_rec[i])
        # porovnani s LOOCV 2D desc
        dve_D_loocv.append(np.round(sum(listik)/82, 2))
        
        
        listik = []
        for i in range(50):
            listik.append(loaded1[0][3][i] == l10ocv_2D_rec[3][i])
        # porovnani s L10OCV 2D desc
        dve_D_l10ocv.append(np.round(sum(listik)/50, 2))
        
        
    print(feat)
    print(dve_D_all)
    D_all.append(dve_D_all)
    print(dve_D_loocv)
    D_all.append(dve_D_loocv)
    print(dve_D_l10ocv)
    D_all.append(dve_D_l10ocv)
    print()
    dve_D_all = [primary[feat], "Recovery of all data D2 [%]"]
    dve_D_loocv = ["", "Recovery of LOOCV D2 [%]"]
    dve_D_l10ocv = ["", "Recovery of L10%OCV D2 [%]"]
    
    



#('IP(A)')
#('EA(A)') this not
#('IP(B)')
#('EA(B)')

#('H(A)') this not
#('L(A)') this not
#('H(B)') this not
#('L(B)') this not

#('r_s(A)')
#('r_p(A)')
#('r_d(A)') this not
#('r_s(B)') this not
#('r_p(B)')
#('r_d(B)') this not

IndexError: list index out of range

In [27]:
D_all

[['IP(A)',
  'Recovery of all data D2 [%]',
  0.74,
  0.65,
  0.31,
  0.18,
  0.0,
  0.0,
  0.0],
 ['', 'Recovery of LOOCV D2 [%]', 0.95, 0.83, 0.05, 0.0, 0.0, 0.0, 0.0],
 ['', 'Recovery of L10%OCV D2 [%]', 0.92, 0.5, 0.36, 0.0, 0.0, 0.0, 0.0],
 ['EA(A)',
  'Recovery of all data D2 [%]',
  0.74,
  0.75,
  0.73,
  0.71,
  0.7,
  0.67,
  0.62],
 ['', 'Recovery of LOOCV D2 [%]', 1.0, 0.99, 0.95, 0.91, 0.83, 0.56, 0.82],
 ['', 'Recovery of L10%OCV D2 [%]', 1.0, 0.98, 0.92, 0.9, 0.8, 0.84, 0.52],
 ['IP(B)',
  'Recovery of all data D2 [%]',
  0.74,
  0.73,
  0.36,
  0.25,
  0.02,
  0.02,
  0.0],
 ['', 'Recovery of LOOCV D2 [%]', 0.98, 0.7, 0.24, 0.33, 0.11, 0.02, 0.02],
 ['', 'Recovery of L10%OCV D2 [%]', 1.0, 0.86, 0.82, 0.36, 0.24, 0.22, 0.22],
 ['EA(B)',
  'Recovery of all data D2 [%]',
  0.75,
  0.71,
  0.6,
  0.45,
  0.39,
  0.42,
  0.14],
 ['', 'Recovery of LOOCV D2 [%]', 0.99, 0.98, 0.49, 0.28, 0.24, 0.26, 0.24],
 ['', 'Recovery of L10%OCV D2 [%]', 1.0, 0.86, 0.88, 0.7, 0.82, 0.5, 0.4

In [29]:
import tabulate
import latextable
print("\nTabulate Latex:")
print(tabulate.tabulate(D_all, headers=["", "", "sigma = 0.001", "sigma = 0.010", "sigma = 0.030", "sigma = 0.05", "sigma = 0.100", "sigma = 0.130", "sigma = 0.300"], tablefmt="latex"))
print("\nTexttable Latex:")
print(latextable.draw_latex(table, caption="A comparison of rocket features."))


Tabulate Latex:
\begin{tabular}{llrrrrrrr}
\hline
        &                             &   sigma = 0.001 &   sigma = 0.010 &   sigma = 0.030 &   sigma = 0.05 &   sigma = 0.100 &   sigma = 0.130 &   sigma = 0.300 \\
\hline
 IP(A)  & Recovery of all data D2 [\%] &            0.74 &            0.65 &            0.31 &           0.18 &            0    &            0    &            0    \\
        & Recovery of LOOCV D2 [\%]    &            0.95 &            0.83 &            0.05 &           0    &            0    &            0    &            0    \\
        & Recovery of L10\%OCV D2 [\%]  &            0.92 &            0.5  &            0.36 &           0    &            0    &            0    &            0    \\
 EA(A)  & Recovery of all data D2 [\%] &            0.74 &            0.75 &            0.73 &           0.71 &            0.7  &            0.67 &            0.62 \\
        & Recovery of LOOCV D2 [\%]    &            1    &            0.99 &            0.95 &           0.

NameError: name 'table' is not defined

In [159]:
loaded = pickle.load( open( "lasso_noise_prim_loocv/sens_data_LOOCV_prim_f_1_n_1.p_gauss", "rb" ) )
loocv_2D_rec = pickle.load( open( "lasso_noise_prim_loocv/LOOCV_descriptors.p", "rb" ) )[1]
li = []
for i in loaded:
    li.append(i[2])
vsechno = np.concatenate(li)
print(vsechno.shape)
print(np.sum(vsechno)/vsechno.shape)

listik = []
for i in range(82):
    listik.append(loaded[0][3][i] == loocv_2D_rec[i])
print(sum(listik)/82)

(4100,)
[0.75512195]
0.9512195121951219


In [157]:
loaded = pickle.load( open( "lasso_noise_prim_loocv/sens_data_LOOCV_prim_f_1_n_2.p", "rb" ) )
loocv_2D_rec = pickle.load( open( "lasso_noise_prim_loocv/LOOCV_descriptors.p", "rb" ) )[1]
li = []
for i in loaded:
    li.append(i[2])
vsechno = np.concatenate(li)
print(vsechno.shape)
print(np.sum(vsechno)/vsechno.shape)

listik = []
for i in range(82):
    listik.append(loaded[0][3][i] == loocv_2D_rec[i])
print(sum(listik)/82)

(4100,)
[0.65195122]
0.8292682926829268


In [171]:
def inicializace_dat():
    Z = np.array([
                    3, 4, 5, 6, 7, 8, 9, 
                    11, 12, 13, 14, 15, 16, 
                    17, 19, 20, 29, 30, 31, 32, 
                    33, 34, 35, 37, 38, 47, 48, 
                    49, 50, 51, 52, 53, 55, 56
    ])
    # atomove cislo prvku A, 34 hodnot
    Prvky = np.array([
                 'Li', 'Be', 'B ', 'C ', 'N ',
                 'O ', 'F ', 'Na', 'Mg', 'Al', 'Si',
                 'P ', 'S ', 'Cl', 'K ', 'Ca', 'Cu',
                 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br',
                 'Rb', 'Sr', 'Ag', 'Cd', 'In', 'Sn',
                 'Sb', 'Te', 'I ', 'Cs', 'Ba'
    ])
    # prvky prislusejici danemu atomovemu cislu, 34 hodnot
    IP = np.array([
                  -5.329, -9.459, -8.190, -10.852, -13.585,
                  -16.433, -19.404, -5.223, -8.037,
                  -5.780, -7.758, -9.751, -11.795,
                  -13.902, -4.433, -6.428, -8.389, -10.136,
                  -5.818, -7.567, -9.262, -10.946, -12.650,
                  -4.289, -6.032, -8.058, -9.581, -5.537,
                  -7.043, -8.468, -9.867, -11.257, -4.006,
                  -5.516
    ])
    # ionizacni potencial (IP)
    EA =  np.array([
        -0.698, 0.631, -0.107, -0.872, -1.867, -3.006, -4.273, -0.716,
        0.693, -0.313, -0.993, -1.920, -2.845, -3.971, -0.621, 0.304,
        -1.638, 1.081, -0.108, -0.949, -1.839, -2.751, -3.739, -0.590,
        0.343, -1.667, 0.839, -0.256, -1.039, -1.847, -2.666, -3.513,
        -0.570, 0.278
    ])
    # elektronova afinita (EA)
    EN = np.array([
        3.014, 4.414, 4.149, 5.862, 7.726, 9.720, 11.839, 2.969, 3.672,
        3.046, 4.375, 5.835, 7.320, 8.936, 2.527, 3.062, 5.014, 4.527,
        2.963, 4.258, 5.551, 6.848, 8.194, 2.440, 2.844, 4.862, 4.371,
        2.897, 4.041, 5.158, 6.266, 7.385, 2.288, 2.619
    ])
    # Elektronegativita dle Mullikenovy definice (EN)
    HOMO = np.array([
        -2.874, -5.600, -3.715, -5.416, -7.239, -9.197, -11.294, -2.819,
        -4.782, -2.784, -4.163, -5.596, -7.106, -8.700, -2.426, -3.864,
        -4.856, -6.217, -2.732, -4.046, -5.341, -6.654, -8.001, -2.360,
        -3.641, -4.710, -5.952, -2.697, -3.866, -4.991, -6.109, -7.236,
        -2.220, -3.346
    ])

    LUMO = np.array([
        -0.978, -2.098, 2.248, 1.992, 3.057, 2.541, 1.251, -0.718, -1.358,
        0.695, 0.440, 0.183, 0.642, 0.574, -0.697, -2.133, -0.641, -1.194,
        0.130, 2.175, 0.064, 1.316, 0.708, -0.705, -1.379, -0.479, -1.309,
        0.368, 0.008, 0.105, 0.099, 0.213, -0.548, -2.129
    ])


    # The radii at which the radial probability density of the valence s, p, 
    # and d orbital are respectively maximal.
    r_s = np.array([
        1.652, 1.078, 0.805, 0.644, 0.539, 0.462, 0.406, 1.715, 1.330, 1.092,
        0.938, 0.826, 0.742, 0.679, 2.128, 1.757, 1.197, 1.099, 0.994, 0.917,
        0.847, 0.798, 0.749, 2.240, 1.911, 1.316, 1.232, 1.134, 1.057, 1.001,
        0.945, 0.896, 2.464, 2.149
    ])

    r_p = np.array([
        1.995, 1.211, 0.826, 0.630, 0.511, 0.427, 0.371, 2.597, 1.897, 1.393,
        1.134, 0.966, 0.847, 0.756, 2.443, 2.324, 1.680, 1.547, 1.330, 1.162,
        1.043, 0.952, 0.882, 3.199, 2.548, 1.883, 1.736, 1.498, 1.344, 1.232,
        1.141, 1.071, 3.164, 2.632
    ])

    r_d = np.array([
        6.930, 2.877, 1.946, 1.631, 1.540, 2.219, 1.428, 6.566, 3.171, 1.939,
        1.890, 1.771, 2.366, 1.666, 1.785, 0.679, 2.576, 2.254, 2.163, 2.373,
        2.023, 2.177, 1.869, 1.960, 1.204, 2.968, 2.604, 3.108, 2.030, 2.065,
        1.827, 1.722, 1.974, 1.351
    ])


    dE = np.array([
        -0.059, -0.038, -0.033, -0.022, 0.430, 0.506, 0.495, 0.466, 1.713,
        1.020, 0.879, 2.638, -0.146, -0.133, -0.127, -0.115, -0.178, -0.087,
        -0.055, -0.005, 0.072, 0.219, 0.212, 0.150, 0.668, 0.275, -0.146,
        -0.165, -0.166, -0.168, -0.266, -0.369, -0.361, -0.350, -0.019,
        0.156, 0.152, 0.203, 0.102, 0.275, 0.259, 0.241, 0.433, 0.341, 0.271,
        0.158, 0.202, -0.136, -0.161, -0.164, -0.169, -0.221, -0.369, -0.375,
        -0.381, -0.156, -0.044, -0.030, 0.037, -0.087, 0.070, 0.083, 0.113,
        0.150, 0.170, 0.122, 0.080, 0.016, 0.581, -0.112, -0.152, -0.158,
        -0.165, -0.095, -0.326, -0.350, -0.381, 0.808, 0.450, 0.264, 0.136,
        0.087
    ])
    # dE = E(RS) - E(ZB) ... 82 hodnot pro binární sloučeniny
    AB = np.array([
        'Li-F ', 'Li-Cl', 'Li-Br', 'Li-I ', 'Be-O ', 'Be-S ', 'Be-Se', 'Be-Te',
        'B -N ', 'B -P ', 'B -As', 'C -C ', 'Na-F ', 'Na-Cl', 'Na-Br', 'Na-I ',
        'Mg-O ', 'Mg-S ', 'Mg-Se', 'Mg-Te', 'Al-N ', 'Al-P ', 'Al-As', 'Al-Sb',
        'Si-C ', 'Si-Si', 'K -F ', 'K -Cl', 'K -Br', 'K -I ', 'Ca-O ', 'Ca-S ',
        'Ca-Se', 'Ca-Te', 'Cu-F ', 'Cu-Cl', 'Cu-Br', 'Cu-I ', 'Zn-O ', 'Zn-S ',
        'Zn-Se', 'Zn-Te', 'Ga-N ', 'Ga-P ', 'Ga-As', 'Ga-Sb', 'Ge-Ge', 'Rb-F ',
        'Rb-Cl', 'Rb-Br', 'Rb-I ', 'Sr-O ', 'Sr-S ', 'Sr-Se', 'Sr-Te', 'Ag-F ',
        'Ag-Cl', 'Ag-Br', 'Ag-I ', 'Cd-O ', 'Cd-S ', 'Cd-Se', 'Cd-Te', 'In-N ',
        'In-P ', 'In-As', 'In-Sb', 'Sn-Sn', 'B -Sb', 'Cs-F ', 'Cs-Cl', 'Cs-Br',
        'Cs-I ', 'Ba-O ', 'Ba-S ', 'Ba-Se', 'Ba-Te', 'Ge-C ', 'Sn-C ', 'Ge-Si',
        'Sn-Si', 'Sn-Ge'
    ])
    # Z vektorů dat vytvoření dictionary obsahující listy, které mají prvky svoje vypočtené hodnoty
    # Kodovani pomoci stringu nazev prvku (dva charaktery dlouhy!!!!)
    oniers = {}
    for i in range(len(Prvky)):
        oniers[Prvky[i]]= [Z[i], IP[i], EA[i], EN[i], HOMO[i], LUMO[i], r_s[i], r_p[i], r_d[i]]

    # Data jednotlivych dimeru ulozenych v dictionary listů, celkem 82 listů
    # Tyto listy jsou vlastně matice o 8 radcich a dvou sloupcich
    dimers = {} # inicializace
    temp = [] #inicializace temporary listu
    for i in AB: # pro kazdy dimer
        for j in range(1,9): # vytvori list listů osmi dvojic hodnot (bez Z[i])
            temp.append( [ oniers[i[:2]][j] , oniers[i[3:]][j] ] )
            # [IP, EA, EN, HOMO, LUMO, r_s, r_p, r_d]
        dimers[i] = temp # kodovani pomoci nazvu dimeru
        temp = [] # clearing pro dalsi iteraci
    dE = dE.reshape(-1,1) # restrukturalizace dat
    return dimers, AB, dE, Z, Prvky, IP, EA, EN, HOMO, LUMO, r_s, r_p, r_d
    
def feature_space_generation(noised_E, noise, noised_feature, sigma, dimers, AB, tier0, tier1, tier2, tier3, tier4, tier5):    
    # Nyni definujeme a nagenerujeme mnoziny moznych deskriptoru
    # Jednotlive mnoziny jsou listy floatovych hodnot ulozene jako dictionary a klic je  nazev dimeru ve formatu '__-__'
    # Brute force definice zakladnich mnozin deskriptorů pro kazdy dimer:
    
    # tier - which tier of the descriptor to include
    # Vektor popisující tvar deskriptorů:
    A1 = {}
    A2 = {}
    A3 = {}
    for i in AB:
        A1[i] = [ dimers[i][0][0] , dimers[i][1][0] , dimers[i][0][1] , dimers[i][1][1] ] 
        A2[i] = [ dimers[i][3][0] , dimers[i][4][0] , dimers[i][3][1] , dimers[i][4][1] ]
        A3[i] = [ dimers[i][5][0] , dimers[i][6][0] , dimers[i][7][0] , dimers[i][5][1] , dimers[i][6][1] , dimers[i][7][1] ]
        
    if noise==True:
        gauss = np.random.normal(1, sigma, 1)[0]
        
        if noised_feature==1 or noised_feature==True:
            for i in AB:
                A1[i][0] = A1[i][0]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==2 or noised_feature==True:
            for i in AB:
                A1[i][1] = A1[i][1]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==3 or noised_feature==True:
            for i in AB:
                A1[i][2] = A1[i][2]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==4 or noised_feature==True:
            for i in AB:
                A1[i][3] = A1[i][3]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==5 or noised_feature==True:
            for i in AB:
                A2[i][0] = A2[i][0]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==6 or noised_feature==True:
            for i in AB:
                A2[i][1] = A2[i][1]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==7 or noised_feature==True:
            for i in AB:
                A2[i][2] = A2[i][2]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==8 or noised_feature==True:
            for i in AB:
                A2[i][3] = A2[i][3]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==9 or noised_feature==True:
            for i in AB:
                A3[i][0] = A3[i][0]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==10 or noised_feature==True:
            for i in AB:
                A3[i][1] = A3[i][1]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==11 or noised_feature==True:
            for i in AB:
                A3[i][2] = A3[i][2]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==12 or noised_feature==True:
            for i in AB:
                A3[i][3] = A3[i][3]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==13 or noised_feature==True:
            for i in AB:
                A3[i][4] = A3[i][4]*np.random.normal(1, sigma, 1)[0]
        if noised_feature==14 or noised_feature==True:
            for i in AB:
                A3[i][5] = A3[i][5]*np.random.normal(1, sigma, 1)[0]
            
        
    DD = []
    DD.append('IP(A)')
    DD.append('EA(A)')
    DD.append('IP(B)')
    DD.append('EA(B)')

    DD.append('H(A)')
    DD.append('L(A)')
    DD.append('H(B)')
    DD.append('L(B)')

    DD.append('r_s(A)')
    DD.append('r_p(A)')
    DD.append('r_d(A)')
    DD.append('r_s(B)')
    DD.append('r_p(B)')
    DD.append('r_d(B)')
    
    if tier0==True:####
        DD_A1=DD[:4]
        DD_A2=DD[4:8]
        DD_A3=DD[8:14]

    # Generovani jednoduchych deskriptoru
    DD_B1 = []
    DD_B2 = []
    DD_B3 = []

    DD_C3 = []
    DD_D3 = []
    DD_E3 = []
    
    if tier1==True:####
        DD_dvojice = list( it.combinations( DD_A1 , 2 ) )
        for j in DD_dvojice:
            DD_B1.append('|'+j[0]+'-'+j[1]+'|')
            DD_B1.append('|'+j[0]+'+'+j[1]+'|')

        DD_dvojice = list( it.combinations( DD_A2 , 2 ) )
        for j in DD_dvojice:
            DD_B2.append('|'+j[0]+'-'+j[1]+'|')
            DD_B2.append('|'+j[0]+'+'+j[1]+'|')

        DD_dvojice = list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_B3.append('|'+j[0]+'-'+j[1]+'|')
            DD_B3.append('|'+j[0]+'+'+j[1]+'|')

    DD = DD + DD_B1 + DD_B2 + DD_B3
    
    if tier1==True:####
        for j in DD_A3:
            DD_C3.append(j+'^2')
            
    if tier2==True:####
        DD_dvojice = list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_C3.append('('+j[0]+'+'+j[1]+')^2')
    
    if tier1==True:####
        for j in DD_A3:
            DD_D3.append('exp('+j+')')
            
    if tier2==True:####
        DD_dvojice = list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_D3.append('exp('+j[0]+'+'+j[1]+')')
            
    if tier2==True:####
        for j in DD_A3:
            DD_E3.append('exp('+j+')^2')

    if tier3==True:
        DD_dvojice = list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_E3.append('exp('+j[0]+'+'+j[1]+')^2')

    DD = DD + DD_C3 + DD_D3 + DD_E3


    B1 = {}
    B2 = {}
    B3 = {}
    C3 = {}
    D3 = {}
    E3 = {}
    temp = []

    for i in AB:

        if tier1==True:####
            dvojice = list( it.combinations( A1[i] , 2 ) )
            for j in dvojice:
                temp.append( abs( j[0] - j[1] ) )
                temp.append( abs( j[0] + j[1] ) )

            B1[i] = temp
            temp = []


            dvojice = list( it.combinations( A2[i] , 2 ) )
            for j in dvojice:
                temp.append( abs( j[0] - j[1] ) )
                temp.append( abs( j[0] + j[1] ) )

            B2[i] = temp
            temp = []


            dvojice=list( it.combinations( A3[i] , 2 ) )
            for j in dvojice:
                temp.append( abs( j[0] - j[1] ) )
                temp.append( abs( j[0] + j[1] ) )

            B3[i] = temp
            temp = []
            
        else:
            
            B1[i] = []
            B2[i] = []
            B3[i] = []
            temp = []



        if tier1==True:####
            for j in A3[i]:
                temp.append( (j)**2 )

        if tier2==True:####
            dvojice=list( it.combinations( A3[i] , 2 ) )
            for j in dvojice:
                temp.append( ( j[0] + j[1] )**2 )

        C3[i] = temp
        temp = []

        if tier1==True:####
            for j in A3[i]:
                temp.append( m.exp( j ) )

        if tier2==True:####
            dvojice = list( it.combinations( A3[i] , 2 ) )
            for j in dvojice:
                temp.append( m.exp( j[0] + j[1] ) )

        D3[i] = temp
        temp = []

        if tier2==True:####
            for j in A3[i]:
                temp.append( m.exp( (j)**2 ) )

        if tier3==True:####
            dvojice = list( it.combinations( A3[i] , 2 ) )
            for j in dvojice:
                temp.append( m.exp( ( j[0] + j[1] )**2 ) )

        E3[i] = temp
        temp = []



    G = {}
    temp = []

    DD_G = []


    # A1,B1 ; A2,B2 lomeno A3,C3,D3,E3
    if tier1==True and tier2==False:####                    
        for j in [DD_A1, DD_A2]:
            for l in [DD_A3]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==False:                    
        for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
            for l in [DD_A3]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])
                    
        for j in [DD_A1, DD_A2]:
            for l in [DD_C3[:6], DD_D3[:6]]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==False:            
        for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
            for l in [DD_A3, DD_C3[:6], DD_D3[:6]]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])
                    
        for j in [DD_A1, DD_A2]:
            for l in [DD_C3[6:], DD_D3[6:]]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==False:                    
        for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
            for l in [DD_A3, DD_C3, DD_D3]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

        for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
            for l in [DD_E3[:6]]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])
                    
        for j in [DD_A1, DD_A2]:
            for l in [DD_E3[6:]]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==True:                       
        for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
            for l in [DD_A3, DD_C3, DD_D3, DD_E3]:
                for k in list(it.product(j,l)):
                    DD_G.append(k[0]+'/'+k[1])

# no tiers:
#    for j in [DD_A1, DD_A2, DD_B1, DD_B2]:
#        for l in [DD_C3, DD_D3, DD_E3]:
#            for k in list(it.product(j,l)):
#                DD_G.append(k[0]+'/'+k[1])
                    

    
    
    
    # A3/D3 a A3/E3
    if tier1==True and tier2==True and tier3==False:
        for j in [DD_D3[:6]]:
            for k in list(it.product(DD_A3,j)):
                DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==False:                
        for j in [DD_D3, DD_E3[:6]]:
            for k in list(it.product(DD_A3,j)):
                DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==True:
        for j in [DD_D3, DD_E3]:
            for k in list(it.product(DD_A3,j)):
                DD_G.append(k[0]+'/'+k[1])

    # B3/D3 a B3/E3
    if tier1==True and tier2==True and tier3==True and tier4==False:
        for j in [DD_D3[:6]]:
            for k in list(it.product(DD_B3,j)):
                DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==False:
        for j in [DD_D3, DD_E3[:6]]:
            for k in list(it.product(DD_B3,j)):
                DD_G.append(k[0]+'/'+k[1])

    elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==True:  
        for j in [DD_D3, DD_E3]:
            for k in list(it.product(DD_B3,j)):
                DD_G.append(k[0]+'/'+k[1])
    
    
# no tiers:
#    for j in [DD_D3, DD_E3]:
#        for k in list(it.product(DD_A3,j)):
#            DD_G.append(k[0]+'/'+k[1])
#
#    for j in [DD_D3, DD_E3]:
#        for k in list(it.product(DD_B3,j)):
#            DD_G.append(k[0]+'/'+k[1])


    # Problemove:

    # A3/A3
    if tier1==True:####
        DD_dvojice=list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_G.append(j[0]+'/'+j[1])
            DD_G.append(j[1]+'/'+j[0])


    # A3/C3
    if tier2==True:####
        DD_dvojice=list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            DD_G.append(j[0]+'/'+'('+j[1]+')'+'^2')
            DD_G.append(j[1]+'/'+'('+j[0]+')'+'^2')
            #DD_G.append(j[0]+'/'+'('+j[0] +'+'+ j[1]+')^2')
            #DD_G.append(j[1]+'/'+'('+j[0] +'+'+ j[1]+')^2')

    if tier3==True:####
        DD_dvojice=list( it.combinations( DD_A3 , 2 ) )
        for j in DD_dvojice:
            #DD_G.append(j[0]+'/'+'('+j[1]+')'+'^2')
            #DD_G.append(j[1]+'/'+'('+j[0]+')'+'^2')
            DD_G.append(j[0]+'/'+'('+j[0] +'+'+ j[1]+')^2')
            DD_G.append(j[1]+'/'+'('+j[0] +'+'+ j[1]+')^2')


    if tier3==True:####
        DD_trojice=list(it.combinations(DD_A3,3))
        for j in DD_trojice:
            for k in [(0,1,2),(1,0,2),(2,0,1)]:
                DD_G.append(j[k[0]]+'/'+'('+j[k[1]] +'+'+ j[k[2]]+')^2')



    # B3/A3:
    if tier2==True:####
        DD_trojice=list(it.combinations(DD_A3,3))
        for j in DD_trojice:
            for k in [(0,1,2),(2,1,0),(0,2,1)]:
                DD_G.append('|'+j[k[0]] +'-'+ j[k[1]]+'| /'+j[k[2]])
    
    if tier2==True:####
        DD_dvojice=list(it.combinations(DD_A3,2))
        for j in DD_dvojice:
            DD_G.append('|'+j[1]+'-'+j[0]+'| /' + j[1])
            DD_G.append('|'+j[0]+'-'+j[1]+'| /' + j[0])

    # B3/C3
    if tier3==True:####
        DD_trojice=list(it.combinations(DD_A3,3))
        for j in DD_trojice:
            for k in [(0,1,2),(2,1,0),(0,2,1)]:
                DD_G.append('|' + j[k[0]] + '-' +  j[k[1]]+'| /'+j[k[2]]+ '^2')
    
    if tier4==True:####
        DD_dvojice=list(it.combinations(DD_A3,2))
        for j in DD_dvojice:
            DD_G.append('|'+j[0]+'-'+j[1]+'| /'+'('+j[0]+'+'+j[1]+')^2')

    if tier3==True:####
        DD_dvojice=list(it.combinations(DD_A3,2))
        for j in DD_dvojice:
            DD_G.append('|'+j[0]+'-'+j[1]+'| /' + j[0]+'^2')
            DD_G.append('|'+j[1]+'-'+j[0]+'| /'+j[1]+'^2')

    if tier4==True:####
        DD_trojice=list(it.combinations(DD_A3,3))
        for j in DD_trojice:
            DD_G.append('|'+j[0]+'-'+j[1]+'| /'+ '('+j[0]+'+'+j[2]+')^2')
            DD_G.append('|'+j[0]+'-'+j[2]+'| /'+ '('+j[0]+'+'+j[1]+')^2')

    if tier4==True:####
        DD_ctverice = list(it.combinations(DD_A3,4))
        for j in DD_ctverice:
            for k in [(0,1,2,3),(0,2,1,3),(0,3,1,2),(2,1,0,3),(3,1,0,2),(2,3,0,1)]:
                #temp.append(abs(j[k[0]]+j[k[1]])/(j[k[2]]+j[k[3]])**2)
                DD_G.append('|'+j[k[0]]+'-'+j[k[1]]+') /'+'('+j[k[2]]+'+'+j[k[3]]+')^2')


    # Zde je TEST: Ciste podily 1/r navic:
    if tier1==True:####
        for j in DD_A3:
            DD_G.append('1/' + j)

    DD =  DD + DD_G

#####################################################
    temp = []
    for i in AB:
        # A1,B1 ; A2,B2 lomeno A3,C3,D3,E3
        if tier1==True and tier2==False:####
            for j in [A1[i], A2[i]]:
                for l in [A3[i]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
        elif tier1==True and tier2==True and tier3==False:
            for j in [A1[i], A2[i], B1[i], B2[i]]:
                for l in [A3[i]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
            for j in [A1[i], A2[i]]:
                for l in [C3[i][:6], D3[i][:6]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
        elif tier1==True and tier2==True and tier3==True and tier4==False:            
            for j in [A1[i], A2[i], B1[i], B2[i]]:
                for l in [A3[i], C3[i][:6], D3[i][:6]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
            for j in [A1[i], A2[i]]:
                for l in [C3[i][6:], D3[i][6:]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
        elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==False:
            for j in [A1[i], A2[i], B1[i], B2[i]]:
                for l in [A3[i], C3[i], D3[i]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
            for j in [A1[i], A2[i], B1[i], B2[i]]:
                for l in [E3[i][:6]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
                        
            for j in [A1[i], A2[i]]:
                for l in [E3[i][6:]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])

        elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==True:   
            for j in [A1[i], A2[i], B1[i], B2[i]]:
                for l in [A3[i], C3[i], D3[i], E3[i]]:
                    for k in list(it.product(j,l)):
                        temp.append(k[0]/k[1])
###################################################
                        
                        
###################################################
        # A3/D3 a A3/E3
        if tier1==True and tier2==True and tier3==False:
            for j in [D3[i][:6]]:
                for k in list(it.product(A3[i],j)):
                    temp.append(k[0]/k[1])
                    
        elif tier1==True and tier2==True and tier3==True and tier4==False:
            for j in [D3[i], E3[i][:6]]:
                for k in list(it.product(A3[i],j)):
                    temp.append(k[0]/k[1])
                
        elif tier1==True and tier2==True and tier3==True and tier4==True:
            for j in [D3[i], E3[i]]:
                for k in list(it.product(A3[i],j)):
                    temp.append(k[0]/k[1])
                
        # B3/D3 a B3/E3
        if tier1==True and tier2==True and tier3==True and tier4==False:
            for j in [D3[i][:6]]:
                for k in list(it.product(B3[i],j)):
                    temp.append(k[0]/k[1])
                    
        elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==False:
            for j in [D3[i], E3[i][:6]]:
                for k in list(it.product(B3[i],j)):
                    temp.append(k[0]/k[1])
                    
        elif tier1==True and tier2==True and tier3==True and tier4==True and tier5==True:  
            for j in [D3[i], E3[i]]:
                for k in list(it.product(B3[i],j)):
                    temp.append(k[0]/k[1])

####################################################
                    
        # PROBLEMOVE PODILY:

        # A3/A3 - vsechny podily jenom ne podily X/X = 1:
        if tier1==True:####
            dvojice=list( it.combinations( A3[i] , 2 ) )
            for j in dvojice:
                temp.append(j[0]/j[1])
                temp.append(j[1]/j[0])


        # A3/C3:
        if tier2==True:####
            dvojice=list(it.combinations(A3[i],2))
            for j in dvojice:
                temp.append(j[0]/(j[1])**2)
                temp.append(j[1]/(j[0])**2)
                #temp.append(j[0]/(j[0] + j[1])**2)
                #temp.append(j[1]/(j[0] + j[1])**2)
        
        if tier3==True:####
            dvojice=list(it.combinations(A3[i],2))
            for j in dvojice:
                #temp.append(j[0]/(j[1])**2)
                #temp.append(j[1]/(j[0])**2)
                temp.append(j[0]/(j[0] + j[1])**2)
                temp.append(j[1]/(j[0] + j[1])**2)                
                
                
        if tier3==True:
            trojice=list(it.combinations(A3[i],3))
            for j in trojice:
                for k in [(0,1,2),(1,0,2),(2,0,1)]:
                    temp.append(j[k[0]]/(j[k[1]] + j[k[2]])**2)


        # B3/A3:
        if tier2==True:
            trojice=list(it.combinations(A3[i],3))
            for j in trojice:
                for k in [(0,1,2),(2,1,0),(0,2,1)]:
                    temp.append(abs(j[k[0]] - j[k[1]])/j[k[2]])

        if tier2==True:
            dvojice=list(it.combinations(A3[i],2))
            for j in dvojice:
                temp.append(abs(1-(j[0]/j[1])))
                temp.append(abs(1-(j[1]/j[0])))


        # B3/C3
        if tier3==True:
            trojice=list(it.combinations(A3[i],3))
            for j in trojice:
                for k in [(0,1,2),(2,1,0),(0,2,1)]:
                    temp.append(abs(j[k[0]] - j[k[1]])/j[k[2]]**2)

        if tier4==True:
            dvojice=list(it.combinations(A3[i],2))
            for j in dvojice:
                temp.append(abs(j[0]-j[1])/(j[0]+j[1])**2)

        if tier3==True:
            dvojice=list(it.combinations(A3[i],2))
            for j in dvojice:
                temp.append(abs(j[0]-j[1])/j[0]**2)
                temp.append(abs(j[1]-j[0])/j[1]**2)

        if tier4==True:
            trojice=list(it.combinations(A3[i],3))
            for j in trojice:
                temp.append(abs(j[0]-j[1])/(j[0]+j[2])**2)
                temp.append(abs(j[0]-j[2])/(j[0]+j[1])**2)

        if tier4==True:
            ctverice = list(it.combinations(A3[i],4))
            for j in ctverice:
                for k in [(0,1,2,3),(0,2,1,3),(0,3,1,2),(2,1,0,3),(3,1,0,2),(2,3,0,1)]:
                    #temp.append(abs(j[k[0]]+j[k[1]])/(j[k[2]]+j[k[3]])**2)
                    temp.append(abs(j[k[0]]-j[k[1]])/(j[k[2]]+j[k[3]])**2)






        # Zde je TEST: Ciste podily 1/r navic:
        if tier1==True:####
            for j in A3[i]:
                temp.append(1/j)


        G[i] = temp
        temp = []



    # F1, F2, F3: ... 44 deskriptoru celkem
    if tier3==True:####
        DD_F1 = []
        for j in list(it.combinations(DD_A1[:2],2)):
            for k in list(it.combinations(DD_A1[2:],2)):
                DD_F1.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "+" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F1.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "-" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F1.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "+" + "|" + k[0] + "+" + k[1] + "|" + "|" )
                DD_F1.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "-" + "|" + k[0] + "+" + k[1] + "|" + "|" )

        DD = DD + DD_F1

        DD_F2 = []
        for j in list(it.combinations(DD_A2[:2],2)):
            for k in list(it.combinations(DD_A2[2:],2)):
                DD_F2.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "+" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F2.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "-" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F2.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "+" + "|" + k[0] + "+" + k[1] + "|" + "|" )
                DD_F2.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "-" + "|" + k[0] + "+" + k[1] + "|" + "|" )

        DD = DD + DD_F2

        DD_F3 = []
        for j in list(it.combinations(DD_A3[:3],2)):
            for k in list(it.combinations(DD_A3[3:],2)):
                DD_F3.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "+" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F3.append( "|" + "|" + j[0] + "-" + j[1] + "|" + "-" + "|" + k[0] + "-" + k[1] + "|" + "|" )
                DD_F3.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "+" + "|" + k[0] + "+" + k[1] + "|" + "|" )
                DD_F3.append( "|" + "|" + j[0] + "+" + j[1] + "|" + "-" + "|" + k[0] + "+" + k[1] + "|" + "|" )
        
        DD = DD + DD_F3

    # F1, F2, F3: ... 52 deskriptoru celkem

    F1 = {}
    F2 = {}
    F3 = {}

    temp = []
    if tier3==True:####
        for i in AB:
            for j in list(it.combinations(A1[i][:2],2)):
                for k in list(it.combinations(A1[i][2:],2)):
                    temp.append( abs( abs(j[0]-j[1]) + abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]-j[1]) - abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) + abs(k[0]+k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) - abs(k[0]+k[1]) ) )

            F1[i] = temp
            temp = []


        for i in AB:
            for j in list(it.combinations(A2[i][:2],2)):
                for k in list(it.combinations(A2[i][2:],2)):
                    temp.append( abs( abs(j[0]-j[1]) + abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]-j[1]) - abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) + abs(k[0]+k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) - abs(k[0]+k[1]) ) )

            F2[i] = temp
            temp = []


        for i in AB:
            for j in list(it.combinations(A3[i][:3],2)):
                for k in list(it.combinations(A3[i][3:],2)):
                    temp.append( abs( abs(j[0]-j[1]) + abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]-j[1]) - abs(k[0]-k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) + abs(k[0]+k[1]) ) )
                    temp.append( abs( abs(j[0]+j[1]) - abs(k[0]+k[1]) ) )

            F3[i] = temp
            temp = []
    else:
        for i in AB:
            F1[i] = []
            F2[i] = []
            F3[i] = []
            temp = []
        
    # 
    Descriptors = len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+ len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]])+len(D3[AB[0]])+len(E3[AB[0]])+len(G[AB[0]])+len(F1[AB[0]])+len(F2[AB[0]])+len(F3[AB[0]])

    # Matice D. Opustíme dictionary a použijeme np.array
    D=np.empty((82,Descriptors),dtype=float)
    for i in range(len(AB)):
        for j in range(len(A1[AB[i]])):
            D[i][j] = A1[AB[i]][j]

        for j in range(len(A2[AB[i]])):
            D[i][j+len(A1[AB[i]])] = A2[AB[i]][j]

        for j in range(len(A3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])] = A3[AB[i]][j]

        for j in range(len(B1[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])] = B1[AB[i]][j]

        for j in range(len(B2[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])] = B2[AB[i]][j]

        for j in range(len(B3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])] = B3[AB[i]][j]

        for j in range(len(C3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])] = C3[AB[i]][j]

        for j in range(len(D3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])] = D3[AB[i]][j]

        for j in range(len(E3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])+len(D3[AB[i]])] = E3[AB[i]][j]

        for j in range(len(G[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])+len(D3[AB[i]])+len(E3[AB[i]])] = G[AB[i]][j]

        for j in range(len(F1[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])+len(D3[AB[i]])+len(E3[AB[i]])+len(G[AB[i]])] = F1[AB[i]][j]

        for j in range(len(F2[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])+len(D3[AB[i]])+len(E3[AB[i]])+len(G[AB[i]])+len(F1[AB[i]])] = F2[AB[i]][j]

        for j in range(len(F3[AB[i]])):
            D[i][j+len(A1[AB[i]])+len(A2[AB[i]])+len(A3[AB[i]])+len(B1[AB[i]])+len(B2[AB[i]])+len(B3[AB[i]])+len(C3[AB[i]])+len(D3[AB[i]])+len(E3[AB[i]])+len(G[AB[i]])+len(F1[AB[i]])+len(F2[AB[i]])] = F3[AB[i]][j]

    #print('A1: ', '0 ... ',len(A1[AB[0]])-1)
    #print('A2: ',len(A1[AB[0]]),' ... ',len(A1[AB[0]])+len(A2[AB[0]])-1)
    #print('A3: ',len(A1[AB[0]])+len(A2[AB[0]]), ' ... ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])-1)
    #print('B1: ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]]), ' ... ',  len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[i]])-1)
    #print('B2: ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]]), ' ... ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])-1)
    #print('B3: ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]]), ' ... ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])-1)
    #print('C3: ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]]),' ... ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]])-1)
    #print('D3: ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]]), ' ... ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]])+len(D3[AB[0]])-1)
    #print('E3: ',len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]])+len(D3[AB[0]]), ' ... ', len(A1[AB[0]])+len(A2[AB[0]])+len(A3[AB[0]])+len(B1[AB[0]])+len(B2[AB[0]])+len(B3[AB[0]])+len(C3[AB[0]])+len(D3[AB[0]])+len(E3[AB[0]])-1)
    if noised_E==True:
        if tier4==False:
            li = []
            for i in AB:
                li.append(np.abs(A3[i][3] - A3[i][4])/np.exp(A3[i][2]+A3[i][3]))
            li = np.array(li)
            D_new = np.empty((D.shape[0], D.shape[1]+1))
            D_new[:,:-1] = D
            D_new[:,-1] = li
            D = D_new
            
        li = []
        for i in AB:
            li.append(np.abs(A3[i][0] - A3[i][5])/np.exp((A3[i][0]+A3[i][5])**2))
        li = np.array(li)
        D_new = np.empty((D.shape[0], D.shape[1]+1))
        D_new[:,:-1] = D
        D_new[:,-1] = li
        D = D_new
        
    return D, DD, A1, A2, A3, B1, B2, B3, C3, D3, E3, F1, F2, F3, G#, DD_A3

def lasso_filter(D, dE, POCET, LAMBDA_LENGTH):
    #D_normalized=preprocessing.normalize(D,axis=0)
    D_normalized=preprocessing.StandardScaler().fit_transform(D)
    LAMBDA=np.empty(LAMBDA_LENGTH,dtype=float)
    LAMBDA[0]=(1/D.shape[0])*np.max((D_normalized.T).dot(dE))
    q=m.pow(1000,1/(LAMBDA_LENGTH-1))
    for i in range(1,len(LAMBDA)):
        LAMBDA[i]=LAMBDA[i-1]/q


    lassocv = linear_model.LassoCV(n_jobs=-1)#(fit_intercept=True, normalize=False, max_iter=1e4, tol=1e-3, copy_X=True)
    #alphas, coefs, _ = lassocv.path(D_normalized, dE, fit_intercept=True, alphas=None, max_iter=1e4, tol=1e-4)
    alphas, coefs, _  = linear_model.Lasso.path(D_normalized, dE, normalize=False, l1_ratio=1, alphas=LAMBDA, max_iter=1e4, tol=1e-4, n_jobs=-1)

    sparse = csr_matrix(coefs[0].T)

    THETA = set()
    for i in range(LAMBDA_LENGTH):
        for el in sparse[i].indices:
            if len(THETA)<POCET:
                THETA.add(el)

    THETA = list(THETA)
    THETA.sort()

    return THETA

def lo(THETA, D, dE):
    # Definujeme nejhorsi mozne MSE, tj. MSE pri coef = 0 a intercept = 0:
    MSE = np.ones((4),dtype = float)*metrics.mean_squared_error(dE, np.zeros(dE.shape, dtype = float)) # ctyr dimenzionalni MSE vektor
    MaxAE = np.zeros((4),dtype = float)
    ND = [ [], [], [], [] ]
    # Specialni rutina pro 1D deskriptor

    for i in THETA:
        OMEGA = D[:,i].reshape(-1, 1)
        ols = linear_model.LinearRegression(fit_intercept = True, normalize = False)
        #OMEGA = preprocessing.StandardScaler().fit_transform(OMEGA)
        ols.fit(OMEGA, dE)
        dE_predicted = ols.predict(OMEGA)
        novy_model_MSE = metrics.mean_squared_error(dE, dE_predicted)

        if novy_model_MSE < MSE[0]:
            MSE[0] = novy_model_MSE
            MaxAE[0] = metrics.max_error(dE,dE_predicted)
            ND[0] = [i, ols.coef_, ols.intercept_, m.sqrt(MSE[0]), MaxAE[0]]

    # Rutina pro 2D, ... ,4D deskriptor
    for j in range(2,5): # 2,3,4
        OMEGA = np.zeros((D.shape[0],j),dtype = float)
        jetice = it.combinations(THETA, j)

        for i in jetice:

            for k in range(j):
                OMEGA[:,k] = D[:,i[k]]

            #OMEGA = preprocessing.StandardScaler().fit_transform(OMEGA)
            ols = linear_model.LinearRegression(fit_intercept = True, normalize = False)
            ols.fit(OMEGA,dE)
            dE_predicted = ols.predict(OMEGA)
            novy_model_MSE = metrics.mean_squared_error(dE,dE_predicted)

            if novy_model_MSE < MSE[j-1]:
                MSE[j-1] = novy_model_MSE
                MaxAE[j-1] = metrics.max_error(dE,dE_predicted)
                ND[j-1] = [i, ols.coef_, ols.intercept_, m.sqrt(MSE[j-1]), MaxAE[j-1]]
    return ND

def cross_validation_LASSO(cross_iter, POCET, LAMBDA_LENGTH, D, dE, test_size=7):
    # Vektory co budou drzet hodnoty pro kazdou cross validaci
    RMSE_CV = np.empty((4,cross_iter),dtype = float)
    MaxAE_CV = np.empty((4,cross_iter),dtype = float)
    recovery = np.zeros((4, cross_iter))
    cross_validation_descriptors = np.zeros((4, cross_iter), dtype=object)

    for cv in range(cross_iter):
        if test_size==1 and cross_iter==82: # LOOCV
            D_CV = D[[i for i in range(cross_iter) if i!=cv], :]
            X_test = np.array([D[cv, :]])
            dE_CV = dE[[i for i in range(cross_iter) if i!=cv], :]
            y_test = np.array(dE[cv, :])
        else: # %CV
            D_CV, X_test, dE_CV, y_test = model_selection.train_test_split(D, dE, test_size=test_size, random_state=cv, shuffle = True)

            
        THETA = lasso_filter(D_CV, dE_CV, POCET, LAMBDA_LENGTH)
        ND = lo(THETA, D_CV, dE_CV)
        
        
        # Pro 1D:
        y_predicted = np.ones((y_test.shape[0],1),dtype = float)*ND[0][2] + np.dot(X_test[:,ND[0][0]].reshape(-1, 1),ND[0][1].T)
        RMSE_CV[0,cv] = np.sqrt(metrics.mean_squared_error(y_test, y_predicted))
        MaxAE_CV[0,cv] = metrics.max_error(y_test, y_predicted)

        #Pro 2D, 3D, 4D:
        for j in range(2,5): # 2,3,4
            temporary = np.zeros((y_test.shape[0],j),dtype = float)
            for k in range(j):
                temporary[:,k] = X_test[:,ND[j-1][0][k]]
            y_predicted = np.ones((y_test.shape[0],1),dtype = float)*ND[j-1][2] + np.dot(temporary,ND[j-1][1].T)
            RMSE_CV[j-1,cv] = np.sqrt(metrics.mean_squared_error(y_test, y_predicted))
            MaxAE_CV[j-1,cv] = metrics.max_error(y_test, y_predicted)

        # column indices of descriptors
        desc = [819, (966, 2717), (966, 2717, 3110), (2151, 2717, 3110, 3399)] # (966, 2717, 3097) (2540, 2717, 3022, 3111)

        if ND[0][0]==desc[0]:
            recovery[0, cv] = 1
        for i in range(1,4):
            if sorted(ND[i][0])==sorted(desc[i]):
                recovery[i, cv] = 1

        cross_validation_descriptors[0, cv] = (ND[0][0])

        for i in range(1,4):
            cross_validation_descriptors[i, cv] = [ND[1][0], ND[2][0], ND[3][0]]

    return RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors

In [47]:
start = time.time()
dimers, AB, dE = inicializace_dat()
D, *_ = feature_space_generation(False, False, 0, 1, dimers, AB, True, True, True, True, True, True)
RMSE_CV, MaxAE_CV, recovery, cross_validation_descriptors = cross_validation_LASSO(82, 30, 100, D, dE, test_size=1)
print((time.time() - start)/60, "min")

56.22204730908076 min


In [212]:
WZ = \
[ 0.011,
0.005,
0.003,
0.002,
0.011,
-0.004,
-0.004,
-0.004,
-0.014,
-0.008,
-0.006,
-0.024,
0.011,
0.007,
0.005,
0.004,
0.030,
0.009,
0.006,
0.002,
0.025,
-0.002,
-0.003,
-0.005,
0.003,
-0.009,
0.010,
0.007,
0.007,
0.006,
0.040,
0.024,
0.020,
0.014,
-0.007,
0.000,
-0.001,
-0.002,
0.008,
-0.002,
-0.004,
-0.005,
0.009,
-0.008,
-0.011,
-0.011,
-0.015,
0.008,
0.007,
0.007,
0.006,
0.035,
0.026,
0.023,
0.017,
0.001,
0.003,
0.002,
0.000,
0.011,
0.002,
-0.001,
-0.004,
0.013,
-0.005,
-0.007,
-0.010,
-0.014,
-0.001,
0.006,
0.006,
0.006,
0.005,
0.018,
0.024,
0.023,
0.019,
0.000,
0.007,
-0.011,
-0.010,
-0.013]

In [211]:
ZA = \
[3,
3,
3,
3,
4,
4,
4,
4,
5,
5,
5,
6,
11,
11,
11,
11,
12,
12,
12,
12,
13,
13,
13,
13,
14,
14,
19,
19,
19,
19,
20,
20,
20,
20,
29,
29,
29,
29,
30,
30,
30,
30,
31,
31,
31,
31,
32,
37,
37,
37,
37,
38,
38,
38,
38,
47,
47,
47,
47,
48,
48,
48,
48,
49,
49,
49,
49,
50,
5,
55,
55,
55,
55,
56,
56,
56,
56,
32,
50,
32,
50,
50]

In [210]:
ZB = \
[9,
17,
35,
53,
8,
16,
34,
52,
7,
15,
33,
6,
9,
17,
35,
53,
8,
16,
34,
52,
7,
15,
33,
51,
6,
14,
9,
17,
35,
53,
8,
16,
34,
52,
9,
17,
35,
53,
8,
16,
34,
52,
7,
15,
33,
51,
32,
9,
17,
35,
53,
8,
16,
34,
52,
9,
17,
35,
53,
8,
16,
34,
52,
7,
15,
33,
51,
50,
51,
9,
17,
35,
53,
8,
16,
34,
52,
6,
6,
14,
14,
32]

In [209]:
A = \
["Li",
"Li",
"Li",
"Li",
"Be",
"Be",
"Be",
"Be",
"B",
"B",
"B",
"C",
"Na",
"Na",
"Na",
"Na",
"Mg",
"Mg",
"Mg",
"Mg",
"Al",
"Al",
"Al",
"Al",
"Si",
"Si",
"K",
"K",
"K",
"K",
"Ca",
"Ca",
"Ca",
"Ca",
"Cu",
"Cu",
"Cu",
"Cu",
"Zn",
"Zn",
"Zn",
"Zn",
"Ga",
"Ga",
"Ga",
"Ga",
"Ge",
"Rb",
"Rb",
"Rb",
"Rb",
"Sr",
"Sr",
"Sr",
"Sr",
"Ag",
"Ag",
"Ag",
"Ag",
"Cd",
"Cd",
"Cd",
"Cd",
"In",
"In",
"In",
"In",
"Sn",
"B",
"Cs",
"Cs",
"Cs",
"Cs",
"Ba",
"Ba",
"Ba",
"Ba",
"Ge",
"Sn",
"Ge",
"Sn",
"Sn"]


In [208]:
B = \
["F",
"Cl",
"Br",
"I",
"O",
"S",
"Se",
"Te",
"N",
"P",
"As",
"C",
"F",
"Cl",
"Br",
"I",
"O",
"S",
"Se",
"Te",
"N",
"P",
"As",
"Sb",
"C",
"Si",
"F",
"Cl",
"Br",
"I",
"O",
"S",
"Se",
"Te",
"F",
"Cl",
"Br",
"I",
"O",
"S",
"Se",
"Te",
"N",
"P",
"As",
"Sb",
"Ge",
"F",
"Cl",
"Br",
"I",
"O",
"S",
"Se",
"Te",
"F",
"Cl",
"Br",
"I",
"O",
"S",
"Se",
"Te",
"N",
"P",
"As",
"Sb",
"Sn",
"Sb",
"F",
"Cl",
"Br",
"I",
"O",
"S",
"Se",
"Te",
"C",
"C",
"Si",
"Si",
"Ge"]

In [213]:
E = \
[-0.059,
-0.038,
-0.033,
-0.022,
0.430,
0.506,
0.495,
0.466,
1.713,
1.020,
0.879,
2.638,
-0.146,
-0.133,
-0.127,
-0.115,
-0.178,
-0.087,
-0.055,
-0.005,
0.072,
0.219,
0.212,
0.150,
0.668,
0.275,
-0.146,
-0.165,
-0.166,
-0.168,
-0.266,
-0.369,
-0.361,
-0.350,
-0.019,
0.156,
0.152,
0.203,
0.102,
0.275,
0.259,
0.241,
0.433,
0.341,
0.271,
0.158,
0.202,
-0.136,
-0.161,
-0.164,
-0.169,
-0.221,
-0.369,
-0.375,
-0.381,
-0.156,
-0.044,
-0.030,
0.037,
-0.087,
0.070,
0.083,
0.113,
0.150,
0.170,
0.122,
0.080,
0.016,
0.581,
-0.112,
-0.152,
-0.158,
-0.165,
-0.095,
-0.326,
-0.350,
-0.381,
0.808,
0.450,
0.264,
0.136,
0.087]

In [41]:
import tabulate
import latextable
import texttable

In [219]:
print("\nTabulate Latex:")
print(tabulate.tabulate(np.array([ZA, ZB, A, B, E, WZ]).T, headers=["Z_A", "Z_B", "A", "B", "E", "WZ"], tablefmt="latex"))
print("\nTexttable Latex:")
print(latextable.draw_latex(table, caption="A comparison of rocket features."))


Tabulate Latex:
\begin{tabular}{rrllrr}
\hline
   Z\_A &   Z\_B & A   & B   &      E &     WZ \\
\hline
     3 &     9 & Li  & F   & -0.059 &  0.011 \\
     3 &    17 & Li  & Cl  & -0.038 &  0.005 \\
     3 &    35 & Li  & Br  & -0.033 &  0.003 \\
     3 &    53 & Li  & I   & -0.022 &  0.002 \\
     4 &     8 & Be  & O   &  0.43  &  0.011 \\
     4 &    16 & Be  & S   &  0.506 & -0.004 \\
     4 &    34 & Be  & Se  &  0.495 & -0.004 \\
     4 &    52 & Be  & Te  &  0.466 & -0.004 \\
     5 &     7 & B   & N   &  1.713 & -0.014 \\
     5 &    15 & B   & P   &  1.02  & -0.008 \\
     5 &    33 & B   & As  &  0.879 & -0.006 \\
     6 &     6 & C   & C   &  2.638 & -0.024 \\
    11 &     9 & Na  & F   & -0.146 &  0.011 \\
    11 &    17 & Na  & Cl  & -0.133 &  0.007 \\
    11 &    35 & Na  & Br  & -0.127 &  0.005 \\
    11 &    53 & Na  & I   & -0.115 &  0.004 \\
    12 &     8 & Mg  & O   & -0.178 &  0.03  \\
    12 &    16 & Mg  & S   & -0.087 &  0.009 \\
    12 &    34 & Mg  & Se  & -0

NameError: name 'table' is not defined

In [186]:
print('Tabulate Table:')
print(tabulate.tabulate(np.array([Z, Prvky, IP, EA, EN, HOMO, LUMO, r_s, r_p, r_d]).T, headers=["Z", "Name", "IP", "EA", "EN", "HOMO", "LUMO", "r_s", "r_p", "r_d"]))
table = texttable.Texttable()
table.set_cols_align(["c"] * 4)
table.set_deco(texttable.Texttable.HEADER | texttable.Texttable.VLINES)
print('\nTexttable Table:')
print(table.draw())

Tabulate Table:
  Z  Name         IP      EA      EN     HOMO    LUMO    r_s    r_p    r_d
---  ------  -------  ------  ------  -------  ------  -----  -----  -----
  3  Li       -5.329  -0.698   3.014   -2.874  -0.978  1.652  1.995  6.93
  4  Be       -9.459   0.631   4.414   -5.6    -2.098  1.078  1.211  2.877
  5  B        -8.19   -0.107   4.149   -3.715   2.248  0.805  0.826  1.946
  6  C       -10.852  -0.872   5.862   -5.416   1.992  0.644  0.63   1.631
  7  N       -13.585  -1.867   7.726   -7.239   3.057  0.539  0.511  1.54
  8  O       -16.433  -3.006   9.72    -9.197   2.541  0.462  0.427  2.219
  9  F       -19.404  -4.273  11.839  -11.294   1.251  0.406  0.371  1.428
 11  Na       -5.223  -0.716   2.969   -2.819  -0.718  1.715  2.597  6.566
 12  Mg       -8.037   0.693   3.672   -4.782  -1.358  1.33   1.897  3.171
 13  Al       -5.78   -0.313   3.046   -2.784   0.695  1.092  1.393  1.939
 14  Si       -7.758  -0.993   4.375   -4.163   0.44   0.938  1.134  1.89
 15  P      

In [191]:
print("\nTabulate Latex:")
print(tabulate.tabulate(np.array([Z, Prvky, IP, EA, EN, HOMO, LUMO, r_s, r_p, r_d]).T, headers=["Z", "Name", "IP", "EA", "EN", "HOMO", "LUMO", "r_s", "r_p", "r_d"], tablefmt="latex"))
print("\nTexttable Latex:")
print(latextable.draw_latex(table, caption="A comparison of rocket features."))


Tabulate Latex:
\begin{tabular}{rlrrrrrrrr}
\hline
   Z & Name   &      IP &     EA &     EN &    HOMO &   LUMO &   r\_s &   r\_p &   r\_d \\
\hline
   3 & Li     &  -5.329 & -0.698 &  3.014 &  -2.874 & -0.978 & 1.652 & 1.995 & 6.93  \\
   4 & Be     &  -9.459 &  0.631 &  4.414 &  -5.6   & -2.098 & 1.078 & 1.211 & 2.877 \\
   5 & B      &  -8.19  & -0.107 &  4.149 &  -3.715 &  2.248 & 0.805 & 0.826 & 1.946 \\
   6 & C      & -10.852 & -0.872 &  5.862 &  -5.416 &  1.992 & 0.644 & 0.63  & 1.631 \\
   7 & N      & -13.585 & -1.867 &  7.726 &  -7.239 &  3.057 & 0.539 & 0.511 & 1.54  \\
   8 & O      & -16.433 & -3.006 &  9.72  &  -9.197 &  2.541 & 0.462 & 0.427 & 2.219 \\
   9 & F      & -19.404 & -4.273 & 11.839 & -11.294 &  1.251 & 0.406 & 0.371 & 1.428 \\
  11 & Na     &  -5.223 & -0.716 &  2.969 &  -2.819 & -0.718 & 1.715 & 2.597 & 6.566 \\
  12 & Mg     &  -8.037 &  0.693 &  3.672 &  -4.782 & -1.358 & 1.33  & 1.897 & 3.171 \\
  13 & Al     &  -5.78  & -0.313 &  3.046 &  -2.784 &  0.6

In [170]:
dimers['Li-F ']

[[-5.329, -19.404],
 [-0.698, -4.273],
 [3.014, 11.839],
 [-2.874, -11.294],
 [-0.978, 1.251],
 [1.652, 0.406],
 [1.995, 0.371],
 [6.93, 1.428]]

In [172]:
dimers, AB, dE, Z, Prvky, IP, EA, EN, HOMO, LUMO, r_s, r_p, r_d = inicializace_dat()

In [176]:
np.array([Z, Prvky, IP, EA, EN, HOMO, LUMO, r_s, r_p, r_d]).T

array([['3', 'Li', '-5.329', '-0.698', '3.014', '-2.874', '-0.978',
        '1.652', '1.995', '6.93'],
       ['4', 'Be', '-9.459', '0.631', '4.414', '-5.6', '-2.098', '1.078',
        '1.211', '2.877'],
       ['5', 'B ', '-8.19', '-0.107', '4.149', '-3.715', '2.248',
        '0.805', '0.826', '1.946'],
       ['6', 'C ', '-10.852', '-0.872', '5.862', '-5.416', '1.992',
        '0.644', '0.63', '1.631'],
       ['7', 'N ', '-13.585', '-1.867', '7.726', '-7.239', '3.057',
        '0.539', '0.511', '1.54'],
       ['8', 'O ', '-16.433', '-3.006', '9.72', '-9.197', '2.541',
        '0.462', '0.427', '2.219'],
       ['9', 'F ', '-19.404', '-4.273', '11.839', '-11.294', '1.251',
        '0.406', '0.371', '1.428'],
       ['11', 'Na', '-5.223', '-0.716', '2.969', '-2.819', '-0.718',
        '1.715', '2.597', '6.566'],
       ['12', 'Mg', '-8.037', '0.693', '3.672', '-4.782', '-1.358',
        '1.33', '1.897', '3.171'],
       ['13', 'Al', '-5.78', '-0.313', '3.046', '-2.784', '0.695',
    